**IMPORTING PACKAGES**

In [19]:
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import re
import geopandas as gpd
import utm
from shapely.geometry import Point, MultiPolygon
from shapely.geometry import Point, Polygon
from shapely.geometry import Point, MultiLineString
from shapely import wkt
import seaborn as sns
#!pip install --upgrade numpy==1.21
from haversine import haversine, Unit
import requests

## Esta librería no te servirá en este análisis, pero
## es la librería básica para scrapear páginas más sencillas

from bs4 import BeautifulSoup as bs
import undetected_chromedriver as uc
import pandas as pd
from io import StringIO
import csv


**BUILDING DATASET**

In [20]:
url_idealista_collected_data_1 = "https://github.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/raw/fb65bda916b55a3f5199eba01688407dc47a1616/Data/Creation_Variables/Idealista_Collected_Data_1.csv"
url_idealista_collected_data_2 = "https://github.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/raw/fb65bda916b55a3f5199eba01688407dc47a1616/Data/Creation_Variables/Idealista_Collected_Data_2.csv"
response = requests.get(url_idealista_collected_data_1)
content = response.content.decode("utf-8")
df1 = pd.read_csv(StringIO(content))
response = requests.get(url_idealista_collected_data_2)
content = response.content.decode("utf-8")
df2 = pd.read_csv(StringIO(content))
data = pd.concat([df1,df2])

In [21]:
print(len(data))
data.drop_duplicates(['id'],inplace=True)
print(len(data))

16323
16301


In [22]:
data.set_index('id', inplace=True)

**RENAME COLUMNS**

## OPEN DATA BARCELONA & TURBOPASSAPI

In [23]:
import csv
url_catalog = "https://raw.githubusercontent.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/18b88c2db2c70842711e50af2e43790f9c1403c3/Data/Sus_Var_Creation/cataleg.csv"

response = requests.get(url_catalog)
content = response.text

d = []
csv_reader = csv.reader(content.splitlines(), delimiter=',')
for row in csv_reader:
    d.append(row)

catalog = pd.DataFrame(d)
catalog.columns = catalog.iloc[0]
catalog.drop(catalog.index[0], inplace=True)
catalog.head(2)

,name,title_ca,title_es,title_en,complementary description,field_definition,openness_score,license_title,date_published,organization_parent_name_ca,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
1,20170706-districtes-barris,Unitats administratives de la ciutat de Barce...,Unidades administrativas de la ciudad de Barce...,Administrative units of the city of Barcelona,S,S,3,Creative Commons Attribution 4.0,2017-07-06,Administració,...,S,N,N,N,N,N,N,N,N,N
2,accessibilitat-via-publica,Accessibilitat a la via pública de la ciutat d...,Accesibilidad en la vía pública de la ciudad d...,Streets' accessibility in Barcelona city,S,S,3,Creative Commons Attribution 4.0,2019-11-15,Territori,...,N,N,N,N,N,N,N,N,N,N


In [24]:
catalog.columns[:100]

Index(['name', 'title_ca', 'title_es', 'title_en', 'complementary description',
       'field_definition', 'openness_score', 'license_title', 'date_published',
       'organization_parent_name_ca', 'organization_name_ca', 'package_count',
       'frequency', 'geolocation', 'token_required', 'historical', 'api',
       'total_views', 'recent_views', 'total_downloads_by_ip',
       'total_downloads_absolute', 'total_api_access_by_api',
       'total_api_access_absolute', 'tags_list', 'fuente', 'author',
       'department', 'url_tornada_ca', 'url_tornada_es', 'url_tornada_en',
       'notes_ca', 'notes_es', 'notes_en', 'url_busqueda_ca',
       'url_busqueda_es', 'url_busqueda_en', 'Agenda_2030_ODS_Principal',
       'Agenda_2030_ODS_Colateral1', 'Agenda_2030_ODS_Colateral2', 'qpj',
       'jpeg', 'prj', 'application/x-7z-compressed', 'qml', 'txt', 'xml',
       'bdf', 'qgz', 'zip', 'wms', 'kml', 'geojson', 'json', 'wmts', 'dwg',
       'ics', 'tif', 'csv', 'kmz', 'xlsx', 'rdf-linked', '

In [25]:
import urllib.request
import json
url = "https://opendata-ajuntament.barcelona.cat/data/en/dataset/est-demo-altes-omissio-edat-quinquenal/resource/d9ccdac7-b708-4d84-942a-a7c48e557a01"
url = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=d9ccdac7-b708-4d84-942a-a7c48e557a01&limit=2'
fileobj = urllib.request.urlopen(url)
print (fileobj.read())

b'{"help": "https://opendata-ajuntament.barcelona.cat/data/api/3/action/help_show?name=datastore_search", "success": true, "result": {"resource_id": "d9ccdac7-b708-4d84-942a-a7c48e557a01", "fields": [{"type": "int4", "id": "_id"}, {"type": "numeric", "id": "Any"}, {"type": "numeric", "id": "Codi_Districte"}, {"type": "text", "id": "Nom_Districte"}, {"type": "numeric", "id": "Codi_Barri"}, {"type": "text", "id": "Nom_Barri"}, {"type": "text", "id": "Edats_quinquennals"}, {"type": "numeric", "id": "Nombre"}], "records": [{"Codi_Districte": "1", "Nom_Districte": "Ciutat Vella", "Edats_quinquennals": "0-4 anys", "Nom_Barri": "el Raval", "Codi_Barri": "1", "Nombre": "52", "_id": 1, "Any": "2021"}, {"Codi_Districte": "1", "Nom_Districte": "Ciutat Vella", "Edats_quinquennals": "5-9 anys", "Nom_Barri": "el Raval", "Codi_Barri": "1", "Nombre": "30", "_id": 2, "Any": "2021"}], "_links": {"start": "/api/action/datastore_search?limit=2&resource_id=d9ccdac7-b708-4d84-942a-a7c48e557a01", "next": "/a

**INTERESTED DATASETS**

In [26]:
for title in range(0,len(catalog)):
    if title < 600:
        pass
    else:
        print(title)
        print(catalog.loc[title, 'title_en'])
        print(catalog.loc[title, 'title_es'])
        print("")
        print("")
    #print(catalog.loc[title, 'name'])
    interest = [34,35,36,39,114,116, 128,129,130,131,133,148,149,155,156,157,168,169,170,172,174,175,176,183,188,191,192,193,194,215,216,217,218,219,220,221,222]
    interest += [224,242,244,245,247,248,249,254,256,267,268,266,269,271,275,278,280,292,309,327,326]
    interest += [341,344,348,347,346,349,386,404,412,441,444,470,475,476,492,493,504,506,509]
    interest += [514,520,521,524,525,526,527,548,]
    additional_check = [105]
    #print(stop after 182)

In [27]:
catalog.columns[:40]

Index(['name', 'title_ca', 'title_es', 'title_en', 'complementary description',
       'field_definition', 'openness_score', 'license_title', 'date_published',
       'organization_parent_name_ca', 'organization_name_ca', 'package_count',
       'frequency', 'geolocation', 'token_required', 'historical', 'api',
       'total_views', 'recent_views', 'total_downloads_by_ip',
       'total_downloads_absolute', 'total_api_access_by_api',
       'total_api_access_absolute', 'tags_list', 'fuente', 'author',
       'department', 'url_tornada_ca', 'url_tornada_es', 'url_tornada_en',
       'notes_ca', 'notes_es', 'notes_en', 'url_busqueda_ca',
       'url_busqueda_es', 'url_busqueda_en', 'Agenda_2030_ODS_Principal',
       'Agenda_2030_ODS_Colateral1', 'Agenda_2030_ODS_Colateral2', 'qpj'],
      dtype='object', name=0)

In [28]:
downloaded = [34,35,36,39,116,128,129,131,133,155,156,157,168,169,175,174,172,176,192,193,194,216,244,245,292,280]
downloaded += [247,248,249,254,267,268,271,309,327,341,347,348,349,412,441,470,444,475,504,514,520,521,525,527]
downloaded += [114]
gpkp_used = [386,493,548]
gkpg_not_used = [404]
not_checked = [130,148,149,170,188,191,183,215,217,218,219,220,221,222,224,216,242,256,266,269,275,278,326,344,346]
not_checked += [476,524,524,526]
not_usable = [492,506,509]
count = 0
for title in interest:
    #if title not in downloaded and title not in not_checked and title not in gpkp_used and title not in gkpg_not_used and title not in not_usable:
    if title in not_checked:    
        print(title)
        print(catalog.loc[title, 'title_en'])
        print(catalog.loc[title, 'title_es'])
        print(catalog.loc[title,'url_busqueda_en'])
        print(catalog.loc[title, 'api'])
        #print(catalog.loc[title, 'url_busqueda_en'])
        print("")
        print("")
        count +=1
    #downloaded = [34,35,36,39]
print(count)

130
Spaces in the city of Barcelona where cinema, theater and concerts take place
Espacios de la ciudad de Barcelona donde se realiza cine, teatro y conciertos
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=culturailleure-cinemesteatresauditoris
Yes


148
Data from the libraries' network of the city of Barcelona
Datos de la red de bibliotecas de la ciudad de Barcelona
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=dades-xarxa-biblioteques-catalunya
Yes


149
People farthest from públic green spaces in the city of Barcelona
Población con menos proximidad a espacios verdes públicos de la ciudad de Barcelona
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=deficit-proximitat-espai-verd
No


170
List of education equipments in the city of Barcelona
Listado de equipamientos de educación de la ciudad de Barcelona
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=equipament-educacio
Yes


183
Scalators in the city of Barce

In [29]:
for title in gpkp_used:
    print(title)
    print(catalog.loc[title, 'title_en'])
    print(catalog.loc[title, 'title_es'])
    print(catalog.loc[title,'url_busqueda_en'])
    print(catalog.loc[title, 'api'])
    #print(catalog.loc[title, 'url_busqueda_en'])
    print("")
    print("")


386
Properties of the city of Barcelona in buildings destined mainly to housing according to number of floors under grazing
Inmuebles de la ciudad de Barcelona en edificios destinados principalmente a vivienda segun numero de  plantas sota rasante
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=immo-edif-hab-segons-num-plantes-sota-rasant
Yes


493
Exposed population to the noise levels from Strategic Noise Map of the city of Barcelona
Población expuesta a los niveles de ruido del Mapa Estratégico de Ruido de la ciudad de Barcelona
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=poblacio-exposada-mapa-estrategic-soroll
Yes


548
Authorizations for ordinary terraces on public use space in the city of Barcelona
Autorizaciones de terrazas ordinarias en espacios de uso público de la ciudad de Barcelona
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=terrasses-comercos-vigents
Yes




In [30]:
for title in downloaded:
    print(title)
    print(catalog.loc[title, 'title_en'])
    print(catalog.loc[title, 'title_es'])
    print(catalog.loc[title,'url_busqueda_en'])
    print(catalog.loc[title, 'api'])
    #print(catalog.loc[title, 'url_busqueda_en'])
    print("")
    print("")


34
Dog friendly areas in the city of Barcelona
Áreas de recreo de perros en la ciudad de Barcelona
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=arees-esbarjo-gossos-bcn
Yes


35
Average gross taxable income per household (€/Year) for the city of Barcelona  
Renda tributaria bruta media por hogar (€/Año) de la ciudad de Barcelona  
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=atles-renda-bruta-per-llar
Yes


36
Average gross taxable income per person (€/Year) for the city of Barcelona  
Renda tributaria bruta media por persona (€/Año) de la ciudad de Barcelona  
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=atles-renda-bruta-per-persona
Yes


39
Average tax income per unit of consumption (€/year) in the city of Barcelona  
Media de la renta tributaria por unidad de consumo (€/Año) en la ciudad de Barcelona  
https://opendata-ajuntament.barcelona.cat/data/en/dataset?q=&name=atles-renda-mitjana
Yes


116
Surface parking spots c

## CODE DISTRICTS

In [31]:
url_polygon_subdistrict = "https://raw.githubusercontent.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/main/Data/Location/BarcelonaCiutat_Barris.csv"
df_polygon_subdistrict = pd.read_csv(url_polygon_subdistrict )
dic_codi_barri = {}
codi_barris = df_polygon_subdistrict.codi_barri.unique()
subdistricts = data.subdistrict.unique()
manual = []
for code in codi_barris:
    nom_barri = df_polygon_subdistrict[df_polygon_subdistrict.codi_barri == code].nom_barri.unique()[0]
    nom_barri = "Subdistrict " + nom_barri
    #print(nom_barri)
    key = '.'.join([element for element in subdistricts if nom_barri.lower() == element.lower()])
    if key == "":
        manual.append(code)
    else:
        dic_codi_barri[code] = key
    #print(key)

In [32]:
dic_codi_barri

{1: 'Subdistrict El Raval',
 3: 'Subdistrict La Barceloneta',
 5: 'Subdistrict El Fort Pienc',
 6: 'Subdistrict La Sagrada Família',
 7: "Subdistrict La Dreta de l'Eixample",
 8: "Subdistrict L'Antiga Esquerra de l'Eixample",
 9: "Subdistrict La Nova Esquerra de l'Eixample",
 10: 'Subdistrict Sant Antoni',
 12: 'Subdistrict La Marina del Prat Vermell',
 14: 'Subdistrict La Font de la Guatlla',
 15: 'Subdistrict Hostafrancs',
 16: 'Subdistrict La Bordeta',
 17: 'Subdistrict Sants - Badal',
 18: 'Subdistrict Sants',
 19: 'Subdistrict Les Corts',
 20: 'Subdistrict La Maternitat i Sant Ramon',
 21: 'Subdistrict Pedralbes',
 23: 'Subdistrict Sarrià',
 24: 'Subdistrict Les Tres Torres',
 25: 'Subdistrict Sant Gervasi - La Bonanova',
 26: 'Subdistrict Sant Gervasi - Galvany',
 27: 'Subdistrict El Putxet i el Farró',
 28: 'Subdistrict Vallcarca i els Penitents',
 29: 'Subdistrict El Coll',
 30: 'Subdistrict La Salut',
 32: "Subdistrict El Camp d'En Grassot i Gràcia Nova",
 33: 'Subdistrict El 

gini_index = pd.read_csv(r"C:\Users\niels\python\UPF\Thesis\Open_data_bcn\2020_atles_renda_index_gini.csv")
codi_barris = gini_index.Codi_Barri.unique()
subdistricts = data.subdistrict.unique()
dic_codi_barri = {}
manual = []
for code in codi_barris:
    nom_barri = gini_index[gini_index.Codi_Barri == code].Nom_Barri.unique()[0]
    nom_barri = "Subdistrict " + nom_barri
    #print(nom_barri)
    key = '.'.join([element for element in subdistricts if nom_barri.lower() == element.lower()])
    if key == "":
        manual.append(code)
    else:
        dic_codi_barri[code] = key
    #print(key)


In [33]:
for code in manual:
    nom_barri = df_polygon_subdistrict[df_polygon_subdistrict.codi_barri == code].nom_barri.unique()[0]
    nom_barri = "Subdistrict " + nom_barri
    print(f'code: {code}, district {nom_barri}')
    if code == 2:
        adjusted_nom_barri = 'Subdistrict El Gòtic'
    if code == 4:
        adjusted_nom_barri = 'Subdistrict Sant Pere - Santa Caterina i la Ribera'
    if code == 11:
        adjusted_nom_barri = 'Subdistrict El Poble Sec - Parc de Montjuïc'
    if code == 13:
        adjusted_nom_barri = 'Subdistrict La Marina del Port'
    if code == 17:
        adjusted_nom_barri = 'Subdistrict Sants - Badal'
    if code == 22:
        adjusted_nom_barri = 'Subdistrict Vallvidrera - El Tibidabo i les Planes'
    if code == 25:
        adjusted_nom_barri = 'Subdistrict Sant Gervasi - La Bonanova'
    if code == 26:
        adjusted_nom_barri = 'Subdistrict Sant Gervasi - Galvany'
    if code == 31:
        adjusted_nom_barri = 'Subdistrict Vila de Gràcia'
    if code == 39:
        adjusted_nom_barri = 'Subdistrict Sant Genís Dels Agudells - Montbau'
    if code == 40:
        adjusted_nom_barri = 'Subdistrict Sant Genís Dels Agudells - Montbau'
    if code == 41:
        adjusted_nom_barri = "Subdistrict La Vall d'Hebron - La Clota"
    if code == 42:
        adjusted_nom_barri = "Subdistrict La Vall d'Hebron - La Clota"
    if code == 46:
        adjusted_nom_barri = 'Subdistrict Can Peguera - El Turó de la Peira'
    if code == 47:
        adjusted_nom_barri = 'Subdistrict Can Peguera - El Turó de la Peira'
    if code == 54:
        adjusted_nom_barri = 'Subdistrict Ciutat Meridiana - Torre Baró - Vallbona'
    if code == 55:
        adjusted_nom_barri = 'Subdistrict Ciutat Meridiana - Torre Baró - Vallbona'
    if code == 56:
        adjusted_nom_barri = 'Subdistrict Ciutat Meridiana - Torre Baró - Vallbona'
    if code == 70:
        adjusted_nom_barri = 'Subdistrict El Besòs'
    #print(f'code: {code}, district {nom_barri} changed to {adjusted_nom_barri}')   
        
    dic_codi_barri[code] = adjusted_nom_barri
    

code: 2, district Subdistrict el Barri Gòtic
code: 4, district Subdistrict Sant Pere, Santa Caterina i la Ribera
code: 11, district Subdistrict el Poble-sec
code: 13, district Subdistrict la Marina de Port
code: 22, district Subdistrict Vallvidrera, el Tibidabo i les Planes
code: 31, district Subdistrict la Vila de Gràcia
code: 39, district Subdistrict Sant Genís dels Agudells
code: 40, district Subdistrict Montbau
code: 41, district Subdistrict la Vall d'Hebron
code: 42, district Subdistrict la Clota
code: 46, district Subdistrict el Turó de la Peira
code: 47, district Subdistrict Can Peguera
code: 54, district Subdistrict Torre Baró
code: 55, district Subdistrict Ciutat Meridiana
code: 56, district Subdistrict Vallbona
code: 70, district Subdistrict el Besòs i el Maresme


In [34]:
dic_codi_barri['FP'] =  "Subdistrict Zona Franca - Port"

In [35]:
codi_barri_unique = df_polygon_subdistrict.codi_barri.unique()
codi_barri_dic = {}
for d in codi_barri_unique:
    nom_barri =df_polygon_subdistrict[df_polygon_subdistrict.codi_barri== d].nom_barri.unique()[0]
    codi_barri_dic[d] = nom_barri
codi_barri_dic['FP'] =  "Zona Franca"

**DEFINING LOCATIONS OF DISTRICTS**

In [36]:
import requests
api_key = "0b9eb838fc774421bda2a98fe630981a"
def opencagedata(dic_codi_barri, api_key):
    dic_sub_geometry = {}
    to_high = []
    not_hit = []
    for key in dic_codi_barri:
        count = 0
        sub = dic_codi_barri[key]
        subdistrict = sub.replace('Subdistrict',"")
        subdistrict = subdistrict.replace(" ", "+")
        subdistrict = subdistrict + "+Barcelona"
        print(subdistrict)
        url = "https://api.opencagedata.com/geocode/v1/json?q="+subdistrict+"&key="+api_key
        print(url)
        response = requests.get(url)
        # Parse the response content to a list of dictionaries
        data = response.json()
        # Parse the response content to a list of dictionaries
        for i in range(data['total_results']):
            check_pass = True
            if data['results'][i]['components']['_category'] == 'place':
                pass
            else: 
                check_pass = False
            if data['results'][i]['components']['_type'] == 'neighbourhood':
                pass
            else:
                check_pass = False
            try: 
                if data['results'][i]['components']['county_code'] == 'B':
                    pass
                else:
                    check_pass = False
            except:
                check_pass = False
            #if data['results'][i]['components']['state'] == 'Catalonia':
            #    pass
            #else:
            #    check_pass = False
            #if data['results'][i]['components']['state_district'] == 'Barcelona':
            #    pass
            #else: 
            #    check_pass = False
            if check_pass == True:
                geometry = data['results'][i]['geometry']
                count += 1
                if count > 2:
                    print(f'Geometry is {geometry} of count: {count}.')
                dic_sub_geometry[key] = geometry
        if key in dic_sub_geometry:
            if count > 1:
                print(f'Count is {count}')
                count = 0
                for i in range(data['total_results']):
                    check_pass = True
                    try: 
                        city = data['results'][i]['components']['city']
                        if city == 'Barcelona':
                            pass
                        else:
                            check_pass = False     
                    except:
                        check_pass = False
                    if data['results'][i]['components']['_category'] == 'place':
                        pass
                    else: 
                        check_pass = False
                    if check_pass == True:
                        count +=1 
                        print(f'New count: {count}')
                        geometry = data['results'][i]['geometry']
                        if count > 1:
                            print(f'Still more than one result {count}')
                            to_high.append(key)
                        dic_sub_geometry[key] = geometry
        else:
            print(f'No results for {sub} with key: {key}')
            not_hit.append(key)
    return dic_sub_geometry, to_high, not_hit
            
                    


In [37]:
lat_long_sub = opencagedata(codi_barri_dic, api_key)

el+Raval+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=el+Raval+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
Count is 2
New count: 1
el+Barri+Gòtic+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=el+Barri+Gòtic+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
la+Barceloneta+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=la+Barceloneta+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
Sant+Pere,+Santa+Caterina+i+la+Ribera+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=Sant+Pere,+Santa+Caterina+i+la+Ribera+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
el+Fort+Pienc+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=el+Fort+Pienc+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
la+Sagrada+Família+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=la+Sagrada+Família+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
la+Dreta+de+l'Eixample+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=la+Dreta+de+l'Eixample+Barcelona&key=0b9eb838fc77442

Sant+Andreu+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=Sant+Andreu+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
la+Sagrera+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=la+Sagrera+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
el+Congrés+i+els+Indians+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=el+Congrés+i+els+Indians+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
Navas+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=Navas+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
el+Camp+de+l'Arpa+del+Clot+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=el+Camp+de+l'Arpa+del+Clot+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
el+Clot+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=el+Clot+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
el+Parc+i+la+Llacuna+del+Poblenou+Barcelona
https://api.opencagedata.com/geocode/v1/json?q=el+Parc+i+la+Llacuna+del+Poblenou+Barcelona&key=0b9eb838fc774421bda2a98fe630981a
la+Vila+Olímpica+del+Poble

In [38]:
lat_long_sub

({1: {'lat': 41.3780596, 'lng': 2.16983},
  2: {'lat': 41.38364, 'lng': 2.17628},
  3: {'lat': 41.3806533, 'lng': 2.1899274},
  4: {'lat': 41.3883219, 'lng': 2.1774107},
  5: {'lat': 41.3959246, 'lng': 2.1823245},
  6: {'lat': 41.4044407, 'lng': 2.1748611},
  7: {'lat': 41.3950373, 'lng': 2.1672069},
  8: {'lat': 41.39, 'lng': 2.155},
  9: {'lat': 41.383389, 'lng': 2.149},
  10: {'lat': 41.3784116, 'lng': 2.1617677},
  11: {'lat': 41.3728061, 'lng': 2.1619718},
  12: {'lat': 41.3538419, 'lng': 2.1423184},
  13: {'lat': 41.3602964, 'lng': 2.1375842},
  14: {'lat': 41.3707824, 'lng': 2.1446756},
  15: {'lat': 41.3750877, 'lng': 2.1429334},
  16: {'lat': 41.37397, 'lng': 2.14377},
  17: {'lat': 41.375466, 'lng': 2.1277096},
  18: {'lat': 41.3753288, 'lng': 2.1349117},
  19: {'lat': 41.385244, 'lng': 2.1328626},
  20: {'lat': 41.3819857, 'lng': 2.1256083},
  21: {'lat': 41.3901401, 'lng': 2.112218},
  22: {'lat': 41.4150674, 'lng': 2.1074824},
  23: {'lat': 41.4021074, 'lng': 2.1151459},
 

**DEFENING NEIGHBOURING DISTRICTS**

In [39]:
def neighboors(lat_long_sub, distance_threshold):
    neighboor_dic_codes = {}
    neighboor_dic_names = {}
    for codi in lat_long_sub[0].keys():
        latitude_loc = lat_long_sub[0][codi]['lat']
        longitude_loc = lat_long_sub[0][codi]['lng']
        neighboors_codes = []
        neighboors_names = []
        for neighboor in lat_long_sub[0].keys():
            latitude_neighboor = lat_long_sub[0][neighboor]['lat']
            longitude_neighboor = lat_long_sub[0][neighboor]['lng']
            distance = haversine((latitude_loc,longitude_loc),(latitude_neighboor,longitude_neighboor))
            if distance < distance_threshold:
                neighboors_codes.append(neighboor)
                neighboors_names.append(dic_codi_barri[neighboor])
        #print(len(neighboors))
        if len(neighboors_codes)< 3:
            print(dic_codi_barri[codi])
        neighboor_dic_codes[codi] = list(set(neighboors_codes))
        neighboor_dic_names[dic_codi_barri[codi]] = list(set(neighboors_names))
    return neighboor_dic_codes, neighboor_dic_names
 

In [40]:
km_15 = neighboors(lat_long_sub, 15)
km_12_5 = neighboors(lat_long_sub, 12.5)
km_10 = neighboors(lat_long_sub, 10)
km_7_5 = neighboors(lat_long_sub, 7.5)
km_5 = neighboors(lat_long_sub, 5)
data['neighboors_15km'] = data['subdistrict'].map(km_15[1])   
data['neighboors_12.5km'] = data['subdistrict'].map(km_12_5[1])   
data['neighboors_10km'] = data['subdistrict'].map(km_10[1])   
data['neighboors_7.5km'] = data['subdistrict'].map(km_7_5[1])  
data['neighboors_5km'] = data['subdistrict'].map(km_5[1])  

**PATHS TO RETRIEVE DATA THAT CANNOT BE DOWNLOADED BY THE API**

In [41]:
BASE_ROUT_MAP = r"C:\Users\niels\python\UPF\Thesis\Open_data_bcn\\"
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
distance_path = "ERROR"

**FUNCTION FOR WEB API/URL**

In [42]:
browser = uc.Chrome()
def from_web_url(url):
    browser.get(url)
    html = browser.page_source
    soup = bs(html, 'lxml')
    file = soup.find("body").text
    data = StringIO(file)
    data = StringIO(file)

    # Use pd.read_csv() to read the CSV-formatted data into a pandas DataFrame
    df = pd.read_csv(data)

    # You can now access your data as a pandas DataFrame
    return df

In [43]:
def from_web_api(resource_id):
    url = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id='+resource_id+'&limit=1000000'
    response = requests.get(url)
    # Parse the response content to a list of dictionaries
    data = response.json()
    data = data['result']['records']
    # Create the Pandas dataframe
    df = pd.DataFrame(data)
    return df
    

### LONGITUDE, LATITUDE SET-UP

In [44]:
data_long_lat = data[['latitude','longitude','subdistrict','neighboors_15km','neighboors_12.5km','neighboors_10km','neighboors_7.5km','neighboors_5km']]
data_unique_id = data_long_lat.index.unique()
data_long_lat.head(5)

,latitude,longitude,subdistrict,neighboors_15km,neighboors_12.5km,neighboors_10km,neighboors_7.5km,neighboors_5km
id,,,,,,,,
31702050,41.391242,2.167659,Subdistrict La Dreta de l'Eixample,"[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict La Nova Esquerra de l'Eixample, S..."
33148486,41.392904,2.160830,Subdistrict La Dreta de l'Eixample,"[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict La Nova Esquerra de l'Eixample, S..."
39158016,41.391495,2.175116,Subdistrict La Dreta de l'Eixample,"[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict La Nova Esquerra de l'Eixample, S..."
40351755,41.387987,2.176783,Subdistrict La Dreta de l'Eixample,"[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict La Nova Esquerra de l'Eixample, S..."
93970881,41.387533,2.167808,Subdistrict La Dreta de l'Eixample,"[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict Les Roquetes, Subdistrict Baró de...","[Subdistrict La Nova Esquerra de l'Eixample, S..."


**FORMULAS DISTANCE TO CALCULATION**

*haver distance formula*

In [45]:
def distance_haver(loc_from, loc_to):
    distance = haversine(loc_from,loc_to)
    #print(distance)
    return distance

*distance for services/ammenities is provided by the longitude/latitude*

In [46]:
def distance_house(max_distance, df_house, house_longitude_col, house_latitude_col,house_neighboor_col, loc_df, loc_longitude_col, loc_latitude_col):
    house_unique_id = df_house.index.unique()
    result_dic = {}
    count = 0
    not_collected = []
    for id_house in house_unique_id:
        close_districts = df_house.loc[id_house][house_neighboor_col]
        house_long = df_house.loc[id_house][house_longitude_col]
        house_lat = df_house.loc[id_house][house_latitude_col]
        house_loc = (house_lat, house_long)
        max_distance = max_distance
        distances = []
       # print(f'Neighboors: {len(close_districts)}')
        #if count > 8000:
            #print(close_districts)
        try:
            sub_loc_df = loc_df[loc_df['subdistrict'].isin(close_districts)].copy(deep=True)
            #print(len(sub_loc_df))
            for i in (sub_loc_df.index):
                loc_long = sub_loc_df.loc[i][loc_longitude_col]
                loc_lat = sub_loc_df.loc[i][loc_latitude_col]
                loc_to = (loc_lat,loc_long)
                distance = distance_haver(house_loc, loc_to) 
                if distance > max_distance:
                    pass
                else:
                    distances.append(distance)    
            result_dic[id_house] = distances
            if count % 5000 == 0:
                #print(count)
                print(f'Data Collected for: {np.round(len(sub_loc_df)/1000)} observations')
            count += 1
        except:
            not_collected.append(id_house)
            print(f'Not collected is: {house_loc}')
    print(f'Number of observation with not collected data is: {len(not_collected)}')
    return result_dic, not_collected

*save results in dic function*

In [47]:
def result_to_dic(distance_path, result_distance, name_loc):
    #writing files
    with open(distance_path+name_loc+"_distance.txt", 'w') as convert_file:
         convert_file.write(json.dumps(result_distance[0]))  
    with open(distance_path+name_loc+'_missing.txt', 'w') as fp:
        for item in result_distance[1]:
            # write each item on a new line
            fp.write("%s\n" % item)
        print('Done')

*function to calculate values if data provided in multipolygon*

In [48]:
#easting, northing, zone_number, zone_letter = utm.from_latlon(latitude, longitude)
from shapely.geometry import Point, Polygon
def check_in_multipolygon(main_df, longitude_col, latitude_col, df_gpkg, var_of_interest):
    from shapely.geometry import Point, Polygon
    house_unique_id = main_df.index.unique()
    result_dic = {}
    no_results = []
    count_id = 0
    for house_id in house_unique_id:
        # Convert latitude and longitude to UTM
        easting, northing, zone_number, zone_letter = utm.from_latlon(main_df.loc[house_id][latitude_col], main_df.loc[house_id][longitude_col])
        point= Point(easting,northing)
        #print(point)
        count  = 0
        inside = False
        for index, row in df_gpkg.iterrows():
            multipolygon = row['geometry']
            if multipolygon.contains(point):
                #print("The point is inside this Multipolygon.")
                count += 1
                #results_dic[i] = df_gpkp[sdf_gkpg.geometry==multipolygon][var_of_interest]
                result_dic[house_id] = row[var_of_interest]
                #print(f"The point is inside this Multipolygon with count: {count}.")
                if count > 1:
                    print(f'strange count higher than 1: {count}')
            else: 
                pass
        if house_id in result_dic:
            pass
        else:
            no_results.append(house_id)
        count_id += 1
        if count_id % 1000 == True:
            print(count_id)
    return result_dic, no_results 

*function to calculate values if data provided in multilinestring*

In [49]:
def check_in_multilinestring(main_df, longitude_col, latitude_col, df_gpkg, var_of_interest, geo_var):
    house_unique_id = main_df.index.unique()
    result_dic = {}
    no_results = []
    count_id = 0
    for house_id in house_unique_id:
        # Convert latitude and longitude to UTM
        easting, northing, zone_number, zone_letter = utm.from_latlon(main_df.loc[house_id][latitude_col], main_df.loc[house_id][longitude_col])
        point= Point(easting,northing)
        count  = 0
        inside = False
        for index, row in df_gpkg.iterrows():
            multilinestring = wkt.loads(row[geo_var])
            if multilinestring.contains(point):
                #print("The point is inside this Multipolygon.")
                count += 1
                #results_dic[i] = df_gpkp[sdf_gkpg.geometry==multipolygon][var_of_interest]
                result_dic[house_id] = row[var_of_interest]
                #print(f"The point is inside this Multipolygon with count: {count}.")
                if count > 1:
                    print(f'strange count higher than 1: {count}')
            else: 
                pass
        if house_id in result_dic:
            pass
        else:
            no_results.append(house_id)
        count_id += 1
        if count_id % 1000 == True:
            print(count_id)
        print(count_id)
    return result_dic, no_results 

*Function to calculate distance from OpenStreetMap Data*

In [50]:
def open_streetmap_distance(max_distance, df_house, house_longitude_col, house_latitude_col,open_street_map_points):
    house_unique_id = df_house.index.unique()
    result_dic = {}
    count = 0
    not_collected = []
    for id_house in house_unique_id:
        house_long = df_house.loc[id_house][house_longitude_col]
        house_lat = df_house.loc[id_house][house_latitude_col]
        house_loc = (house_lat, house_long)
        max_distance = max_distance
        distances = []
        for open_street_map_point in open_street_map_points:
            distance = distance_haver(house_loc, open_street_map_point)
            if distance > max_distance:
                pass
            else:
                distances.append(distance)
        if len(distances) > 0:
            result_dic[id_house] = distances
        else:
            not_collected.append(house_id)
        count += 1
        if count%1000 == True:
            print(np.round(count/1000))
    print(f'Number of observation with not collected data is: {len(not_collected)}')
    return result_dic, not_collected

**SET-UP IF WEIGHTED AVERAGE BY INHABITANTS NEED TO BE CALCULATED**

In [51]:
resource_id = "109daad7-72f5-47a9-a4f0-22f85e5d094c"
inhabitants_subdistrict_df= from_web_api(resource_id)
print(inhabitants_subdistrict_df.columns)
inhabitants_subdistrict_df['Nombre'] = inhabitants_subdistrict_df['Nombre'].astype(float)
inhabitants_subdistrict_df = inhabitants_subdistrict_df.groupby('Codi_Barri').sum()
inhabitants_subdistrict_df.reset_index(inplace=True)
inhabitants_subdistrict_df['Codi_Barri'] = inhabitants_subdistrict_df['Codi_Barri'].astype(int)
inhabitants_subdistrict_df['subdistrict'] = inhabitants_subdistrict_df['Codi_Barri'].map(dic_codi_barri)
inhabitants_subdistrict_df.head(3)
inhabitants_subdistrict_df['WA_share'] = 2
for index, row in inhabitants_subdistrict_df.iterrows():
    subdistrict = row['subdistrict']
    nombre = row['Nombre']
    sub_df = inhabitants_subdistrict_df[inhabitants_subdistrict_df['subdistrict']==subdistrict]
    som = sub_df.Nombre.sum()
    inhabitants_subdistrict_df.loc[index, 'WA_share'] = nombre / inhabitants_subdistrict_df[inhabitants_subdistrict_df['subdistrict']==subdistrict]['Nombre'].sum()

Index(['Any_alta_padro', 'Codi_Districte', 'Sexe', 'Nom_Districte',
       'Nom_Barri', 'Codi_Barri', 'Nombre', '_id', 'Any'],
      dtype='object')


# SUSTAINABLE DATA COLLECTION

## SAVE RESULTS ON PC:


In [52]:
save_sustainable_variables = False
#save path for distance variables
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
#save path for neighboorhood variables
area_path = r"C:\Users\niels\python\UPF\Thesis\Area_stats\\"
#save path for data provided in gpkp files containing multipolygon or multilinestring specific data
gpkp_path = r"C:\Users\niels\python\UPF\Thesis\gpkg\\"

## ECOLOGICAL DIMENSION

**BUSSTOPS**

In [53]:
resource_id = "2d190658-93ac-4c43-a23f-c5d313b1ae9c"
busstops_df = from_web_api(resource_id)
busstops_df["LATITUD"] = busstops_df["LATITUD"].astype(float)
busstops_df["LONGITUD"] = busstops_df["LONGITUD"].astype(float)
columns = ["_id",'LATITUD',"LONGITUD","BARRI"]
busstops_df = busstops_df[columns]
busstops_df['BARRI'] = busstops_df['BARRI'].astype(int)
busstops_df['subdistrict'] = busstops_df['BARRI'].map(dic_codi_barri)
busstops_df.head(3)


,_id,LATITUD,LONGITUD,BARRI,subdistrict
0,1,41.380067,2.155939,10,Subdistrict Sant Antoni
1,2,41.405856,2.146179,27,Subdistrict El Putxet i el Farró
2,3,41.430801,2.174516,46,Subdistrict Can Peguera - El Turó de la Peira


In [ ]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
bus_distance = distance_house(3, data_long_lat,'longitude','latitude','neighboors_5km',busstops_df,'LONGITUD',"LATITUD")
if save_sustainable_variables:
    result_to_dic(distance_path, bus_distance, 'bus')

Data Collected for: 3.0 observations
Not collected is: (41.3803548, 2.1524542)
Not collected is: (41.3811974, 2.1549069)
Not collected is: (41.3828762, 2.1519372)
Not collected is: (41.391568, 2.1481999)
Not collected is: (41.380312, 2.1537497)
Not collected is: (41.3815054, 2.154199)
Not collected is: (41.3831439, 2.1531168)
Not collected is: (41.3866609, 2.1481689)
Not collected is: (41.3836913, 2.1536539)
Not collected is: (41.3884798, 2.1502616)
Not collected is: (41.3837077, 2.1548263)
Not collected is: (41.3823484, 2.1543777)
Not collected is: (41.3831488, 2.1512597)
Not collected is: (41.3865597, 2.1430774)
Not collected is: (41.3835112, 2.1548595)
Not collected is: (41.3811446, 2.1548388)
Not collected is: (41.3805916, 2.1515747)
Not collected is: (41.3878092, 2.1449321)
Not collected is: (41.382874, 2.1532685)
Not collected is: (41.3833207, 2.1463526)
Not collected is: (41.3851015, 2.1491949)
Not collected is: (41.3853492, 2.1472601)
Not collected is: (41.3793048, 2.1532029)


**HIGHWAY**

In [126]:
overpass_url = "http://overpass-api.de/api/interpreter"
highway_query = """[out:json];
way["highway"]["maxspeed"="10"](41.237543901082574,2.010841369628906,41.41054830178558,2.3006057739257812);
out geom;"""
count = 0
for i in range(100,125,5):
    print(f'Speed Checked For Highway is: {i}')
    response = requests.get(overpass_url, 
                        params={'data': highway_query.replace("10", str(i))})
    data_overpass = response.json()
    if count == 0:
        df_highway = pd.DataFrame.from_dict(data_overpass['elements'])
    else:
        df_highway = pd.concat([df_highway, pd.DataFrame.from_dict(data_overpass['elements'])])
    count += 1
df_highway.set_index('id', inplace= True)
points_highway = []

Speed Checked For Highway is: 100
Speed Checked For Highway is: 105
Speed Checked For Highway is: 110
Speed Checked For Highway is: 115
Speed Checked For Highway is: 120


In [127]:
points_highway = []
for i in df_highway.index:
    b= df_highway.loc[i].geometry
    for way in b:
        #print(way)
        #geometry = way.get("geometry", [])
        #for point in geometry:
         #   lat = point.get("lat")
          #  lon = point.get("lon")
           # #print(f"Latitude: {lat}, Longitude: {lon}")
        points_highway.append((way['lat'],way['lon']))
points_highway[:2]

[(41.3267505, 2.0786446), (41.3256939, 2.0778395)]

In [128]:
df_highway.head(5)

,type,bounds,nodes,geometry,tags
id,,,,,
16359173,way,"{'minlat': 41.3256939, 'minlon': 2.0778395, 'm...","[213053416, 208680744]","[{'lat': 41.3267505, 'lon': 2.0786446}, {'lat'...","{'highway': 'motorway', 'lanes': '3', 'lit': '..."
16359249,way,"{'minlat': 41.3191811, 'minlon': 2.0727873, 'm...","[178536644, 208648833, 208675985]","[{'lat': 41.3191811, 'lon': 2.0727873}, {'lat'...","{'highway': 'motorway', 'lanes': '2', 'lit': '..."
19883843,way,"{'minlat': 41.272323, 'minlon': 2.0411166, 'ma...","[205263720, 2260624517, 2260624522, 2260624549...","[{'lat': 41.272323, 'lon': 2.0411166}, {'lat':...","{'highway': 'trunk', 'lanes': '2', 'maxspeed':..."
19925899,way,"{'minlat': 41.272256, 'minlon': 2.0400546, 'ma...","[205270767, 205270768, 205270769]","[{'lat': 41.2724365, 'lon': 2.0410897}, {'lat'...","{'bicycle': 'yes', 'highway': 'trunk', 'lanes'..."
20147068,way,"{'minlat': 41.3268567, 'minlon': 2.0789686, 'm...","[213052903, 178536646]","[{'lat': 41.3268567, 'lon': 2.0789686}, {'lat'...","{'bridge': 'yes', 'highway': 'motorway', 'lane..."


In [130]:
highway_distance = open_streetmap_distance(20, data_long_lat, 'longitude','latitude',points_highway)
if save_sustainable_variables:
    result_to_dic(distance_path, highway_distance, 'highway')

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
Number of observation with not collected data is: 0


**METRO**

In [546]:
overpass_url = "http://overpass-api.de/api/interpreter"
subway_query = """[out:json];
node["public_transport"="station"]["station"="subway"](41.31830206651102,2.053070068359375,41.44941741390757,2.2961425781249996);
out center;"""
response = requests.get(overpass_url, 
                        params={'data': subway_query})
data_overpass = response.json()
df_metro = pd.DataFrame.from_dict(data_overpass['elements'])
df_metro.head()
points_metro = []
for i in df_metro.index:
    points_metro.append((df_metro.loc[i]['lat'],df_metro.loc[i]['lon']))
points_metro[:2]

[(41.3301838, 2.1370899), (41.3237458, 2.1331165)]

In [547]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
metro_distance = open_streetmap_distance(20, data_long_lat, 'longitude','latitude',points_metro)
if save_sustainable_variables:
    result_to_dic(distance_path, metro_distance, 'metro')

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
Number of observation with not collected data is: 0
Done


**TRAIN**

In [131]:
overpass_url = "http://overpass-api.de/api/interpreter"
train_query = """[out:json];
node["public_transport"="station"]['train'='yes'](41.281934557995356,2.0159912109375,41.46691378485316,2.2848129272460938);
out center;"""
response = requests.get(overpass_url, 
                        params={'data': train_query})
data_overpass = response.json()
df_train = pd.DataFrame.from_dict(data_overpass['elements'])
df_train.head()
points_train = []
for i in df_train.index:
    points_train.append((df_train.loc[i]['lat'],df_train.loc[i]['lon']))
points_train[:2]

[(41.3642387, 2.100942), (41.3988657, 2.1251389)]

In [132]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
train_distance = open_streetmap_distance(20, data_long_lat, 'longitude','latitude',points_train)
if save_sustainable_variables:
    result_to_dic(distance_path, train_distance, 'train')

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
Number of observation with not collected data is: 0


## ENVIRONMENTAL

**PARKS AND (BOTANIC) GARDENS**

In [133]:
url = "https://www.bcn.cat/tercerlloc/files/cultura/opendatabcn_cultura_parcs-i-jardins.csv"
parks_and_gardens_loc_df =from_web_url(url)
parks_and_gardens_loc_df= parks_and_gardens_loc_df[['register_id','name', 'institution_name','addresses_neighborhood_id','secondary_filters_name', 'geo_epgs_4326_x','geo_epgs_4326_y']]
print(parks_and_gardens_loc_df.secondary_filters_name.unique())
park_garden = ['Parcs i jardins']
park_and_gardens_loc_df = parks_and_gardens_loc_df[parks_and_gardens_loc_df.secondary_filters_name.isin(park_garden)]
park_and_gardens_loc_df['addresses_neighborhood_id'] =park_and_gardens_loc_df['addresses_neighborhood_id'].astype(int)
park_and_gardens_loc_df['subdistrict'] = park_and_gardens_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
print(len(park_and_gardens_loc_df))
park_and_gardens_loc_df.head(3)
#botanic gardens are included in dataset

EmptyDataError: No columns to parse from file

In [551]:
park_and_garden_distance = distance_house(5, data_long_lat,'longitude','latitude','neighboors_5km',park_and_gardens_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(gpkp_path,park_and_garden_distance, 'park_and_garden')

0
284
500
284
1000
284
1500
258
2000
272
2500
279
3000
253
3500
247
4000
247
4500
248
5000
248
5500
265
6000
207
6500
237
7000
138
7500
167
8000
223
8500
159
9000
217
9500
235
10000
281
10500
173
11000
283
11500
288
12000
227
12500
227
13000
178
13500
93
14000
272
14500
268
15000
228
15500
117
16000
182
Number of observation with not collected data is: 0
Done


**NEIGHBOORHOUD AREA SIZE**

In [148]:
resource_id = "bb402991-6226-4b33-a901-7d23843ec9e1"
neighboorhoodsize_df = from_web_api(resource_id)
neighboorhoodsize_df['Superfície (ha)'] = neighboorhoodsize_df['Superfície (ha)'].astype(float)
neighboorhoodsize_df['Codi_Barri'] = neighboorhoodsize_df['Codi_Barri'].astype(int)
neighboorhoodsize_df['subdistrict'] = neighboorhoodsize_df['Codi_Barri'].map(dic_codi_barri)
print(len(neighboorhoodsize_df.subdistrict.unique()))
neighboorhoodsize_df = neighboorhoodsize_df.groupby('subdistrict').sum()
neighboorhoodsize_df.reset_index(inplace=True)
neighboorhoodsize_df.head(3)
if save_sustainable_variables:
        neighboorhoodsize_df.to_excel(area_path+"neighborhood_area_size"+'.xlsx')

68


**VIEWPOINTS**

In [159]:
resource_id = "50e946a8-f4a2-4a16-8b40-6553bf4ccd81"
viewpoints_loc_df = from_web_api(resource_id)
columns = ['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']
viewpoints_loc_df = viewpoints_loc_df[columns]
print(viewpoints_loc_df.secondary_filters_name.unique())
viewpoints_loc_df['addresses_neighborhood_id'] =viewpoints_loc_df['addresses_neighborhood_id'].astype(int)
viewpoints_loc_df['geo_epgs_4326_x'] =viewpoints_loc_df['geo_epgs_4326_x'].astype(float)
viewpoints_loc_df['geo_epgs_4326_y'] =viewpoints_loc_df['geo_epgs_4326_y'].astype(float)
viewpoints_loc_df['subdistrict'] = viewpoints_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
print(len(viewpoints_loc_df))
viewpoints_loc_df.head(2)

['' 'Parcs i jardins']
15


,register_id,name,institution_name,secondary_filters_name,addresses_neighborhood_id,geo_epgs_4326_x,geo_epgs_4326_y,subdistrict
0,﻿92094121416,Mirador,Torre de Collserola,,22,41.415863,2.111786,Subdistrict Vallvidrera - El Tibidabo i les Pl...
1,﻿95125123610,Parc del Mirador del Poble-sec,,Parcs i jardins,11,41.371921,2.170183,Subdistrict El Poble Sec - Parc de Montjuïc


In [161]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
viewpoint_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km',viewpoints_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,viewpoint_distance, 'viewpoint')


Done


def check_in_multilinestring(main_df, longitude_col, latitude_col, df_gpkg, var_of_interest, geo_var):
    house_unique_id = main_df.index.unique()
    result_dic = {}
    no_results = []
    count_id = 0
    for house_id in house_unique_id:
        #print(f'House id: {house_id}')
        # Convert latitude and longitude to UTM
        easting, northing, zone_number, zone_letter = utm.from_latlon(main_df.loc[house_id][latitude_col], main_df.loc[house_id][longitude_col])
        point= Point(easting,northing)
        count  = 0
        inside = False
        for index, row in df_gpkg.iterrows():
            multilinestring = wkt.loads(row[geo_var])
            #print(f'Multilinestring is: {multilinestring}')
            if multilinestring.intersects(point):
                print("The point is inside this Multipolygon.")
                count += 1
                #results_dic[i] = df_gpkp[sdf_gkpg.geometry==multipolygon][var_of_interest]
                result_dic[house_id] = 2 #row[var_of_interest]
                #print(f"The point is inside this Multipolygon with count: {count}.")
                if count > 1:
                    print(f'strange count higher than 1: {count}')
            else: 
                pass
        if house_id in result_dic:
            pass
        else:
            no_results.append(house_id)
        count_id += 1
        if count_id % 1000 == True:
            print(count_id)
    return result_dic, no_results 

import math



def calculate_distance(point1, point2):
    # Euclidean distance formula
    distance = math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2 + (point2[2] - point1[2])**2)
    return distance / 1000

def calculate_distance_between_multilinestring_and_point(multilinestring, point):
    # Convert the point coordinates from latitude and longitude to projected coordinates
    #lon, lat = point[:2]
    # Perform the necessary conversion here according to your projected coordinate system
    #projected_point = (lon, lat)  # Assign the projected coordinates to x and y

    # Create Shapely Point object from the projected point coordinates
    #point = Point(projected_point)

    # Create Shapely MultiLineString object from the MULTILINESTRING Z coordinates
    print(f'Point is {point}')
    print(f'Multistring is {multilinestring}')
    #linestrings = []
    #for line_segment in multilinestring:
    #    linestring = LineString([(x, y, z) for x, y, z in line_segment])
    #    linestrings.append(linestring)
    #multilinestring = MultiLineString(linestrings)

    # Find the nearest point on the MultiLineString to the given point
    nearest_point = nearest_points(multilinestring, point)[0]
    print(f'Nearest point is {nearest_point}')
    # Calculate the distance between the nearest point and the given point
    distance = calculate_distance(point.y, point.x, nearest_point.y, nearest_point.x)
    print(f'Distance is {distance}')
    def distance_haver(loc_from, loc_to):
        print(f'Loc from {loc_from}')
        print(f'Loc to {loc_to}')
        distance = haversine(loc_from,loc_to)
        #print(distance)
        return distance
    distance = distance_haver(point, nearest_point)
    print(f'Distance is {distance}')
    return distance
main_df = data
#house_unique_id = main_df.index.unique()
house_unique_id = [31702050]
for house_id in house_unique_id:
    print(house_id)
    longitude_col = 'longitude'
    latitude_col = 'latitude'
    easting, northing, zone_number, zone_letter = utm.from_latlon(main_df.loc[house_id][latitude_col], main_df.loc[house_id][longitude_col])
    point= Point(easting,northing)
    print(point)
    count  = 0
    inside = False
    point = loc_from
    for index, row in noise_pollution_day_df.iterrows():
        multilinestring_z_str = row['geometry']
        loc_to = multilinestring_z_str
        x1, y1 = loc_from
        #x2, y2 = loc_to
        print(multilinelinestring)
        distance = calculate_distance(loc_from, loc_to)
        print(distance)
       



from shapely.wkt import loads

# MultiLineString Z representation as a string
multilinestring_z_str = "MULTILINESTRING Z ((430229.788593716 4586585.199057749 0, 430181.8765840912 4586593.704435924 0))"

# Parse the MultiLineString Z string into a MultiLineString object
multilinestring_z = loads(multilinestring_z_str)

# Extract the coordinates from the MultiLineString object
coordinates = list(multilinestring_z.coords)

print(coordinates)

import math
from shapely.geometry import Point, LineString, MultiLineString
from shapely.ops import nearest_points

def calculate_distance(x1, y1, x2, y2):
    # Euclidean distance formula
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance / 1000  # Convert distance from meters to kilometers

# Coordinates of the point
point_coordinates = (430414.1355977786, 4582524.356556184)
point = Point(point_coordinates)

# Coordinates of the MultiLineString
multilinestring_coordinates = [(430229.788593716, 4586585.199057749, 0),
                               (430181.8765840912, 4586593.704435924, 0)]

# Create LineString object from the coordinates
linestring = LineString(multilinestring_coordinates)

# Create MultiLineString object from the LineString
multilinestring = MultiLineString([linestring])

# Find the nearest point on the MultiLineString to the given point
nearest_point = nearest_points(multilinestring, point)[0]

# Calculate the distance between the nearest point and the given point
distance = calculate_distance(point.x, point.y, nearest_point.x, nearest_point.y)
print(distance)

import math

def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert coordinates from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c  # Earth's radius in kilometers

    return distance

# Coordinates of the 'Loc' points
loc_from = (4582524.356556184, 430414.1355977786)
loc_to = (4586585.199057749, 430229.788593716)

# Extract latitude and longitude from the points
lat1, lon1 = loc_from
lat2, lon2 = loc_to[:2]

# Calculate the distance between the two points
distance = calculate_distance(lat1, lon1, lat2, lon2)
print(distance)

**NOISE POLLUTION NIGHTIME**

noise_pollution_night = gpd.read_file(BASE_ROUT_MAP+"2017_soroll_nit_pob.gpkg")
noise_pollution_night_df = pd.DataFrame(noise_pollution_night)
print(noise_pollution_night_df.columns)
noise_pollution_night_df.head(3)

print(len(noise_pollution_night_df))

**NOISE POLLUTION**

resource_id = "ac364e89-e43e-4081-a607-e0438938aa48"
noise_pollution_df = from_web_api(resource_id)
noise_pollution_df.head(4)

print(len(noise_pollution_df))

def check_in_multilinestring(main_df, longitude_col, latitude_col, df_gpkg, var_of_interest, geo_var):
    house_unique_id = main_df.index.unique()
    result_dic = {}
    no_results = []
    count_id = 0
    for house_id in house_unique_id:
        #print(f'House id: {house_id}')
        # Convert latitude and longitude to UTM
        easting, northing, zone_number, zone_letter = utm.from_latlon(main_df.loc[house_id][latitude_col], main_df.loc[house_id][longitude_col])
        point= Point(easting,northing)
        count  = 0
        inside = False
        for index, row in df_gpkg.iterrows():
            multilinestring = wkt.loads(row[geo_var])
            #print(f'Multilinestring is: {multilinestring}')
            if multilinestring.intersects(point):
                print("The point is inside this Multipolygon.")
                count += 1
                #results_dic[i] = df_gpkp[sdf_gkpg.geometry==multipolygon][var_of_interest]
                result_dic[house_id] = 2 #row[var_of_interest]
                #print(f"The point is inside this Multipolygon with count: {count}.")
                if count > 1:
                    print(f'strange count higher than 1: {count}')
            else: 
                pass
        if house_id in result_dic:
            pass
        else:
            no_results.append(house_id)
        count_id += 1
        if count_id % 1000 == True:
            print(count_id)
    return result_dic, no_results 

from shapely.geometry import Point, MultiLineString

# Define a MultiLineString object
multilinestring = MultiLineString([[(430229.788593716, 4586585.19905775), (430181.876584091, 4586593.70443592)]])

# Define a point object
point = Point(430201.401, 4586589.987)

# Test if the point intersects with the MultiLineString
if multilinestring.intersects(point):
    print("The point intersects with the MultiLineString")
else:
    print("The point does not intersect with the MultiLineString")


old= 0
step = 100
for i in range(0,17000, step):
    i = i + step
    noise_pollution_gpkg= check_in_multilinestring(data_long_lat[old:i], 'longitude', 'latitude', noise_pollution_df, 'Valor','GEOM_WKT')
    noise_path = r"C:\Users\niels\python\UPF\Thesis\noise_pollution\\"
    naam = 'noiso_pollution_'+str(old)+'_'+str(i)
    result_to_dic(noise_path,noise_pollution_gpkg, naam)
    old = i

**NOISE POLLUTION 2**

resource_id = "ac364e89-e43e-4081-a607-e0438938aa48"
noise_pollution_df = from_web_api(resource_id)
noise_pollution_df.head(4)

ac364e89-e43e-4081-a607-e0438938aa48

**HEAT IMPACT**

*level of heat impact*

In [134]:
url = "https://github.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/blob/ceb279b7f497b339bdb646e517a04716fc89e726/Data/Sus_Var_Creation/confort_termic_od.gpkg?raw=true"
response = requests.get(url)

with open("confort_termic_od.gpkg", "wb") as file:
    file.write(response.content)

level_of_heat_comfort = gpd.read_file("confort_termic_od.gpkg")


#level_of_heat_comfort = gpd.read_file(BASE_ROUT_MAP+"confort_termic_od.gpkg")
level_of_heat_comfort_df = pd.DataFrame(level_of_heat_comfort)
print(level_of_heat_comfort_df.columns)
level_of_heat_comfort_df.head(4)

Index(['gridcode', 'geometry'], dtype='object')


,gridcode,geometry
0,5,"MULTIPOLYGON (((431549.921 4590773.386, 431550..."
1,4,"MULTIPOLYGON (((430819.921 4590178.177, 430819..."
2,6,"MULTIPOLYGON (((431589.921 4590593.177, 431589..."
3,3,"MULTIPOLYGON (((431654.921 4589673.177, 431654..."


level_of_heat_comfort = gpd.read_file(BASE_ROUT_MAP+"confort_termic_od.gpkg")

In [135]:
level_of_heat_comfort_gpkg= check_in_multipolygon(data_long_lat[:], 'longitude', 'latitude', level_of_heat_comfort_df, 'gridcode')
if save_sustainable_variables:
    result_to_dic(gpkp_path,level_of_heat_comfort_gpkg, 'level_of_heat_comfort')

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001


*vulnerability factors heatimpact*

In [107]:
url = "https://github.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/blob/ceb279b7f497b339bdb646e517a04716fc89e726/Data/Sus_Var_Creation/2017_factors_vulnera.gpkg?raw=true"
response = requests.get(url)
with open("2017_factors_vulnera.gpkg", "wb") as file:
    file.write(response.content)

vulnerable_factors_heatimpact = gpd.read_file("2017_factors_vulnera.gpkg")
vulnerable_factors_heatimpact_df = pd.DataFrame(vulnerable_factors_heatimpact)
print(vulnerable_factors_heatimpact_df.columns)
vulnerable_factors_heatimpact_df.head(3)

Index(['Rec_CP1', 'geometry'], dtype='object')


,Rec_CP1,geometry
0,4,"MULTIPOLYGON (((431304.850 4580691.507, 431258..."
1,4,"MULTIPOLYGON (((431050.990 4581133.590, 431058..."
2,4,"MULTIPOLYGON (((430889.560 4580894.594, 430884..."


vulnerable_factors_heatimpact = gpd.read_file(BASE_ROUT_MAP+"2017_factors_vulnera.gpkg")
vulnerable_factors_heatimpact_df = pd.DataFrame(vulnerable_factors_heatimpact)
print(vulnerable_factors_heatimpact_df.columns)
vulnerable_factors_heatimpact_df.head(3)

In [108]:
vulnerable_factors_heatimpact_gpkg= check_in_multipolygon(data_long_lat, 'longitude', 'latitude', vulnerable_factors_heatimpact_df,'Rec_CP1')
gpkp_path = r"C:\Users\niels\python\UPF\Thesis\gpkg\\"
if save_sustainable_variables:
    result_to_dic(gpkp_path,vulnerable_factors_heatimpact_gpkg, 'vulnerable_factor_heatimpact')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

*vulnerable_to_heatimpact*

In [403]:
url = 'https://github.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/blob/b9f6b8a304b0b19e2ca503194ba4965220faa81e/Data/Sus_Var_Creation/ate_vulnera_0-75_od.gpkg'
response = requests.get(url)
with open("ate_vulnera_0-75_od.gpkg, "wb") as file:
    file.write(response.content)

vulnerable_to_heatimpact = gpd.read_file("ate_vulnera_0-75_od.gpkg)
vulnerable_to_heatimpact_df = pd.DataFrame(vulnerable_to_heatimpact)
print(vulnerable_to_heatimpact_df.columns)
vulnerable_to_heatimpact_df.head(3)

Index(['nRisc', 'geometry'], dtype='object')


,nRisc,geometry
0,1A,"MULTIPOLYGON (((430974.921 4580812.134, 430974..."
1,1A,"MULTIPOLYGON (((430069.725 4581268.177, 430069..."
2,1A,"MULTIPOLYGON (((430769.921 4581006.921, 430769..."


In [409]:
vulnerable_to_heatimpact_gpkg= check_in_multipolygon(data_long_lat, 'longitude', 'latitude', vulnerable_to_heatimpact_df, 'nRisc')
gpkp_path = r"C:\Users\niels\python\UPF\Thesis\gpkg\\"
if save_sustainable_variables:
    result_to_dic(gpkp_path,vulnerable_to_heatimpact_gpkg, 'vulnerable_to_heatimpact')

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
Done


*CHECK AREA*

#easting, northing, zone_number, zone_letter = utm.from_latlon(latitude, longitude)
def check_in_multipolygon(longitude, latitude, df_gpkg):
    # Convert latitude and longitude to UTM
    easting, northing, zone_number, zone_letter = utm.from_latlon(latitude, longitude)
    point= Point(easting,northing)
    count  = 0
    inside = False
    for multipolygon in df_gpkg.geometry.unique():
        # Check if the Point is inside the Multipolygon
        #print(multipolygon)
        if multipolygon.contains(point):
            print("The point is inside this Multipolygon.")
            count += 1
            inside = True
        else:
            pass
            #print("The point is outside this Multipolygon.")
    return inside, count, latitude, longitude 
latitude_list = data.latitude.values
longitude_list = data.longitude.values
latitude_vulnerable_to_heat_impact = []
longitude_vulnerable_to_heat_impact = []
latitude_density_to_heat_impact = []
longitude_density_to_heat_impact = []
latitude_people_furthest_from_green_space = []
longitude_people_furthest_from_green_space = []
for i in range(len(latitude_list)):
    if i%1000 == True:
        print(i)
    latitude = latitude_list[i]
    longitude = longitude_list[i]
    vulnerable_to_heat_impact_result = check_in_multipolygon(longitude,latitude, vulnerable_to_heatimpact_df)
    if vulnerable_to_heat_impact_result[0] == True:
        latitude_vulnerable_to_heat_impact.append( vulnerable_to_heat_impact_result[2])
        longitude_vulnerable_to_heat_impact.append( vulnerable_to_heat_impact_result[3])
    density_to_heat_impact_result = check_in_multipolygon(longitude,latitude, density_to_heatimpact_df)
    if density_to_heat_impact_result[0] == True:
        latitude_density_to_heat_impact.append( density_to_heat_impact_result[2])
        longitude_density_to_heat_impact.append( density_to_heat_impact_result[3])
    people_furthest_from_green_space_result = check_in_multipolygon(longitude,latitude, people_furthest_from_green_space_df)
    if people_furthest_from_green_space_result[0] == True:
        latitude_people_furthest_from_green_space.append( people_furthest_from_green_space_result[2])
        longitude_people_furthest_from_green_space.append( people_furthest_from_green_space_result[3])
    

In [ ]:
len(latitude_density_to_heat_impact)

def check_in_multipolygon(longitude, latitude, df_gpkg):
    # Convert latitude and longitude to UTM
    easting, northing, zone_number, zone_letter = utm.from_latlon(latitude, longitude)
    point= Point(easting,northing)
    count = 0
    for multipolygon in df_gpkg.geometry.unique():
    
        # Check if the Point is inside the Multipolygon
        #print(multipolygon)
        if multipolygon.contains(point):
            print(point)
            print("The point is inside this Multipolygon.")
            count += 1
        else:
            point = np.NaN
            #print("The point is outside this Multipolygon.")
    if count > 0:
        pass
    else:
        pass
    
latitude_list = data.latitude.values
longitude_list = data.longitude.values
inside = 0
vulnerable_to_heatimpact_points = []
density_to_heatimpact_points= []
people_furthest_from_green_space_points = []
for i in range(len(latitude_list)):
    
    latitude = latitude_list[i]
    longitude = longitude_list[i]
    check_in_multipolygon(longitude,latitude, vulnerable_to_heatimpact_df)
    #check_in_multipolygon(longitude,latitude, density_to_heatimpact_df, density_to_heatimpact_points)
    #check_in_multipolygon(longitude,latitude, people_furthest_from_green_space_df,people_furthest_from_green_space_points )

**DISTANCE TO THE BEACH**

In [203]:
overpass_url = "http://overpass-api.de/api/interpreter"
beach_query = '''/*
This has been generated by the overpass-turbo wizard.
The original search was:
“beach”
*/
[out:json];
(
  rel[natural=beach](41.35902953579532,2.18353271484375,41.444527878381756,2.2443008422851567);
);
out geom;'''
response = requests.get(overpass_url, 
                    params={'data': beach_query})
data_overpass = response.json()
df_beach = pd.DataFrame.from_dict(data_overpass['elements'])
points_beach = []
for i in df_beach.index:
    b= df_beach.loc[i].members
    for way in b:
        geometry = way.get("geometry", [])
        for point in geometry:
            lat = point.get("lat")
            lon = point.get("lon")
            #print(f"Latitude: {lat}, Longitude: {lon}")
            points_beach.append((lat,lon))
points_beach[:2]

[(41.4144737, 2.2309319), (41.4148071, 2.2322603)]

In [553]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
beach_distance = open_streetmap_distance(20, data_long_lat, 'longitude','latitude',points_beach)
if save_sustainable_variables:
    result_to_dic(distance_path, beach_distance, 'beach')

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
Number of observation with not collected data is: 0
Done


### SOCIAL DIMENSION

**POLICE STATIONS**

In [387]:
resource_id = "40ab2784-3555-466a-996c-55f3d900c0e5"
police_stations_loc_df = from_web_api(resource_id)
print(police_stations_loc_df.columns)
#police_stations_loc_df = pd.read_csv(r"C:\Users\niels\python\UPF\Thesis\Open_data_bcn\hadministracio_comissaries-de-policia.csv")
police_stations_loc_df = police_stations_loc_df[['register_id','name','addresses_neighborhood_id','geo_epgs_4326_x','geo_epgs_4326_y']]
police_stations_loc_df.head(2)
police_stations_loc_df['addresses_neighborhood_id'] = police_stations_loc_df['addresses_neighborhood_id'].astype(int)
police_stations_loc_df['geo_epgs_4326_x'] =police_stations_loc_df['geo_epgs_4326_x'].astype(float)
police_stations_loc_df['geo_epgs_4326_y'] =police_stations_loc_df['geo_epgs_4326_y'].astype(float)
police_stations_loc_df['subdistrict'] = police_stations_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
print(len(police_stations_loc_df))
police_stations_loc_df.head(2)

Index(['addresses_roadtype_name', 'addresses_end_street_number',
       'values_attribute_name', 'addresses_road_name', 'values_category',
       'addresses_zip_code', 'secondary_filters_id', 'values_value',
       'addresses_town', 'geo_epgs_4326_y', 'geo_epgs_4326_x',
       'secondary_filters_name', 'secondary_filters_tree',
       'addresses_district_name', 'geo_epgs_25831_x',
       'addresses_start_street_number', 'register_id', 'institution_id',
       'addresses_main_address', 'addresses_district_id',
       'addresses_roadtype_id', 'addresses_type', 'addresses_neighborhood_id',
       '_id', 'name', 'addresses_road_id', 'created', 'geo_epgs_25831_y',
       'institution_name', 'modified', 'secondary_filters_asia_id',
       'secondary_filters_fullpath', 'values_description', 'values_id',
       'addresses_neighborhood_name', 'values_outstanding',
       'values_attribute_id'],
      dtype='object')
30


,register_id,name,addresses_neighborhood_id,geo_epgs_4326_x,geo_epgs_4326_y,subdistrict
0,﻿99400146277,Comissaria de Policia de la Generalitat - Moss...,60,41.444820,2.190379,Subdistrict Sant Andreu
1,﻿95353140047,Unitat Territorial de la Guàrdia Urbana del Di...,48,41.436529,2.170144,Subdistrict La Guineueta


In [388]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
police_stations_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km',police_stations_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path, police_stations_distance, 'police_stations')

0
30
500
30
1000
30
1500
30
2000
30
2500
30
3000
30
3500
30
4000
30
4500
30
5000
30
5500
30
6000
30
6500
30
7000
30
7500
30
8000
30
8500
28
9000
30
9500
30
10000
30
10500
30
11000
30
11500
30
12000
30
12500
30
13000
30
13500
29
14000
30
14500
30
15000
30
15500
29
16000
30
Number of observation with not collected data is: 0
Done


**BARS**

In [136]:
url = "https://www.bcn.cat/tercerlloc/files/cultura/opendatabcn_cultura_espais-de-musica-i-copes.csv"
music_and_drinks_loc_df = from_web_url(url)
print(music_and_drinks_loc_df.columns)
music_and_drinks_loc_df= music_and_drinks_loc_df[['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']]
print(music_and_drinks_loc_df.secondary_filters_name.unique())
bar = ['Bars i pubs musicals','Discoteques','Cocteleries', 'Xampanyeries','Karaokes','Tablaos flamencs']
music_and_drinks_loc_df['geo_epgs_4326_x'] = music_and_drinks_loc_df['geo_epgs_4326_x'].astype(float)
music_and_drinks_loc_df['geo_epgs_4326_y'] =music_and_drinks_loc_df['geo_epgs_4326_y'].astype(float)
music_and_drinks_loc_df['subdistrict'] =music_and_drinks_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
bar_loc_df = music_and_drinks_loc_df[music_and_drinks_loc_df.secondary_filters_name.isin(bar)]


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.135)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A2A813+48355]
	(No symbol) [0x009BC4B1]
	(No symbol) [0x008C5358]
	(No symbol) [0x008B7D96]
	(No symbol) [0x008B7AB9]
	(No symbol) [0x008C67C0]
	(No symbol) [0x0091C4D8]
	(No symbol) [0x0090A536]
	(No symbol) [0x008E82DC]
	(No symbol) [0x008E93DD]
	GetHandleVerifier [0x00C8AABD+2539405]
	GetHandleVerifier [0x00CCA78F+2800735]
	GetHandleVerifier [0x00CC456C+2775612]
	GetHandleVerifier [0x00AB51E0+616112]
	(No symbol) [0x009C5F8C]
	(No symbol) [0x009C2328]
	(No symbol) [0x009C240B]
	(No symbol) [0x009B4FF7]
	BaseThreadInitThunk [0x76327D59+25]
	RtlInitializeExceptionChain [0x7742B74B+107]
	RtlClearBits [0x7742B6CF+191]


In [ ]:

bar_distance = distance_house(3, data_long_lat,'longitude','latitude','neighboors_5km', bar_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path, bar_distance, 'bar')

**RESTAURANTS**

In [417]:
url = "https://www.bcn.cat/tercerlloc/files/restaurants/opendatabcn_restaurants_restaurants-csv.csv"
restaurant_loc_df = from_web_url(url)
restaurant_loc_df= restaurant_loc_df[['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id','geo_epgs_4326_x','geo_epgs_4326_y']]
print(restaurant_loc_df.secondary_filters_name.unique())
restaurant_loc_df['geo_epgs_4326_x'] = restaurant_loc_df['geo_epgs_4326_x'].astype(float)
restaurant_loc_df['geo_epgs_4326_y'] = restaurant_loc_df['geo_epgs_4326_y'].astype(float)
restaurant_loc_df['subdistrict'] = restaurant_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
restaurant = ['Restaurants']
restaurant_loc_df = restaurant_loc_df[restaurant_loc_df.secondary_filters_name.isin(restaurant)]

['Restaurants' 'Cocteleries' 'Xampanyeries' 'Karaokes' 'Tablaos flamencs'
 'Discoteques' 'Bars i pubs musicals' 'Teatres']


In [418]:

restaurant_distance = distance_house(3, data_long_lat,'longitude','latitude','neighboors_5km', restaurant_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path, restaurant_distance, 'restaurant')

0
2436
500
2436
1000
2436
1500
2347
2000
2405
2500
2458
3000
2420
3500
2384
4000
2384
4500
2393
5000
2393
5500
2413
6000
2268
6500
2230
7000
1309
7500
1772
8000
2305
8500
1797
9000
2218
9500
2269
10000
2461
10500
1364
11000
2447
11500
2478
12000
2192
12500
2192
13000
1584
13500
279
14000
2392
14500
2399
15000
2085
15500
433
16000
1829
Number of observation with not collected data is: 0
Done


**EDUCATION**

In [421]:
url = "https://www.bcn.cat/tercerlloc/files/llista-equipaments/opendatabcn_llista-equipaments_educacio-csv.csv"
all_education_loc_df = from_web_url(url)
all_education_loc_df= all_education_loc_df[['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']]
all_education_loc_df['geo_epgs_4326_x'] = all_education_loc_df['geo_epgs_4326_x'].astype(float)
all_education_loc_df['geo_epgs_4326_y'] = all_education_loc_df['geo_epgs_4326_y'].astype(float)
all_education_loc_df['subdistrict'] = all_education_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
print(all_education_loc_df.secondary_filters_name.unique())
under_three_education = ['Ensenyament infantil (0-3 anys)']
under_three_education_loc_df =  all_education_loc_df[all_education_loc_df.secondary_filters_name.isin(under_three_education)]
three_to_six_education = ['Ensenyament infantil (3-6 anys)']
three_to_six_education_loc_df = all_education_loc_df[all_education_loc_df.secondary_filters_name.isin(three_to_six_education)]
primary_school= ['Educació primària']
primary_school_loc_df = all_education_loc_df[all_education_loc_df.secondary_filters_name.isin(primary_school)]
secundary_school = ['Educació secundària']
secundary_school_loc_df = all_education_loc_df[all_education_loc_df.secondary_filters_name.isin(secundary_school)]
university = ['Universitats']
university_loc_df = all_education_loc_df[all_education_loc_df.secondary_filters_name.isin(university)]

[nan 'Ensenyament infantil (0-3 anys)' 'Escoles Bressol municipals'
 'Ensenyament infantil (3-6 anys)' 'Educació primària'
 'Educació secundària' 'Formació professional' 'Música' 'Universitats'
 'Idiomes' 'Informàtica' 'Adults' 'WiFi BCN' 'Restaurants'
 'Altres esports' 'Instal·lacions esportives' "Sales d'estudi" 'CAPs'
 'Altres' 'Grans establiments' 'Habitatges tutelats' 'Gimnàstica' 'Ioga'
 'Fitness' 'Biblioteques' 'Residències gent gran' 'Discoteques']


*under three education*

In [422]:
under_three_education_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', under_three_education_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path, under_three_education_distance, 'under_three_education')

0
294
500
294
1000
294
1500
294
2000
294
2500
294
3000
294
3500
294
4000
294
4500
294
5000
294
5500
294
6000
294
6500
294
7000
294
7500
294
8000
294
8500
284
9000
294
9500
294
10000
294
10500
294
11000
294
11500
294
12000
294
12500
294
13000
294
13500
286
14000
294
14500
294
15000
294
15500
292
16000
294
Number of observation with not collected data is: 0
Done


*three to six education*

In [423]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
three_to_six_education_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', three_to_six_education_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,three_to_six_education_distance, 'three_to_six_education')

0
346
500
346
1000
346
1500
346
2000
346
2500
346
3000
346
3500
346
4000
346
4500
346
5000
346
5500
346
6000
346
6500
346
7000
346
7500
346
8000
346
8500
331
9000
346
9500
346
10000
346
10500
346
11000
346
11500
346
12000
346
12500
346
13000
346
13500
337
14000
346
14500
346
15000
346
15500
346
16000
346
Number of observation with not collected data is: 0
Done


*primary school*

In [424]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
primary_school_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', primary_school_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,primary_school_distance, 'primary_school')

0
336
500
336
1000
336
1500
336
2000
336
2500
336
3000
336
3500
336
4000
336
4500
336
5000
336
5500
336
6000
336
6500
336
7000
336
7500
336
8000
336
8500
321
9000
336
9500
336
10000
336
10500
336
11000
336
11500
336
12000
336
12500
336
13000
336
13500
327
14000
336
14500
336
15000
336
15500
336
16000
336
Number of observation with not collected data is: 0
Done


*secundary school*

In [426]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
secundary_school_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', secundary_school_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,secundary_school_distance, 'secundary_school')

0
235
500
235
1000
235
1500
235
2000
235
2500
235
3000
235
3500
235
4000
235
4500
235
5000
235
5500
235
6000
235
6500
235
7000
235
7500
235
8000
235
8500
225
9000
235
9500
235
10000
235
10500
235
11000
235
11500
235
12000
235
12500
235
13000
235
13500
232
14000
235
14500
235
15000
235
15500
235
16000
235
Number of observation with not collected data is: 0
Done


*university*

In [427]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
university_distance = distance_house(15, data_long_lat,'longitude','latitude','neighboors_15km', university_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,university_distance, 'university')

0
248
500
248
1000
248
1500
248
2000
248
2500
248
3000
248
3500
248
4000
248
4500
248
5000
248
5500
248
6000
248
6500
248
7000
248
7500
248
8000
248
8500
248
9000
248
9500
248
10000
248
10500
248
11000
248
11500
248
12000
248
12500
248
13000
248
13500
248
14000
248
14500
248
15000
248
15500
248
16000
248
Number of observation with not collected data is: 0
Done


**PHARMACY & HOSPITAL/CLINIUES**

In [236]:
resource_id = "b89329ea-81ea-496c-9cd0-6758336e6c20"
pharmacy_loc_df = from_web_api(resource_id)
columns = ['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']
pharmacy_loc_df = pharmacy_loc_df[columns]
pharmacy_loc_df['addresses_neighborhood_id'] = pharmacy_loc_df['addresses_neighborhood_id'].astype(int)
pharmacy_loc_df['geo_epgs_4326_x'] = pharmacy_loc_df['geo_epgs_4326_x'].astype(float)
pharmacy_loc_df['geo_epgs_4326_y'] = pharmacy_loc_df['geo_epgs_4326_y'].astype(float)
pharmacy_loc_df['subdistrict'] = pharmacy_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
print(pharmacy_loc_df.secondary_filters_name.unique())
print(len(pharmacy_loc_df))
pharmacy_loc_df.head(3)

['Farmàcies' "Farmàcies servei de 9 a 22 h tot l'any"
 'Farmàcies permanents']
1132


,register_id,name,institution_name,secondary_filters_name,addresses_neighborhood_id,geo_epgs_4326_x,geo_epgs_4326_y,subdistrict
0,﻿75990043657,Farmàcia Farran Briera,,Farmàcies,16,41.370047,2.136400,Subdistrict La Bordeta
1,﻿75990017007,Farmàcia Mercadé Salavert,,Farmàcies,65,41.408173,2.186392,Subdistrict El Clot
2,﻿75990017227,Farmàcia Carbó Pallarés,,Farmàcies,38,41.422057,2.142471,Subdistrict La Teixonera


In [237]:
resource_id = "9e135848-eb0a-4bc5-8e60-de558213b3ed"
hospital_and_cliniques_loc_df = from_web_api(resource_id)
columns = ['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']
print(len(hospital_and_cliniques_loc_df))
hospital_and_cliniques_loc_df = hospital_and_cliniques_loc_df[columns]
hospital_and_cliniques_loc_df['addresses_neighborhood_id'] = hospital_and_cliniques_loc_df['addresses_neighborhood_id'].astype(int)
hospital_and_cliniques_loc_df['geo_epgs_4326_y'] = hospital_and_cliniques_loc_df['geo_epgs_4326_y'].astype(float)
hospital_and_cliniques_loc_df['geo_epgs_4326_x'] = hospital_and_cliniques_loc_df['geo_epgs_4326_x'].astype(float)
hospital_and_cliniques_loc_df['subdistrict'] = hospital_and_cliniques_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
hospital_and_cliniques = [ 'Hospitals i clíniques','Centres urgències (CUAPs)', 'CAPs']
hospital_and_cliniques_loc_df = hospital_and_cliniques_loc_df[hospital_and_cliniques_loc_df.secondary_filters_name.isin(hospital_and_cliniques)]
print(hospital_and_cliniques_loc_df.secondary_filters_name.unique())
print(len(hospital_and_cliniques_loc_df))
hospital_and_cliniques_loc_df.head(3)

141
['Hospitals i clíniques' 'CAPs' 'Centres urgències (CUAPs)']
139


,register_id,name,institution_name,secondary_filters_name,addresses_neighborhood_id,geo_epgs_4326_x,geo_epgs_4326_y,subdistrict
0,﻿179151955,Hospital de dia - Numància,,Hospitals i clíniques,18,41.381424,2.141044,Subdistrict Sants
1,﻿99281083228,Comunitat Terapèutica Clínica Llúria,,Hospitals i clíniques,7,41.392499,2.159835,Subdistrict La Dreta de l'Eixample
2,﻿92086031184,Centre d'Atenció Primària Ciutat Meridiana,,CAPs,54,41.460414,2.181428,Subdistrict Ciutat Meridiana - Torre Baró - Va...


def distance_house(max_distance, df_house, house_longitude_col, house_latitude_col,house_neighboor_col, loc_df, loc_longitude_col, loc_latitude_col):
    house_unique_id = df_house.index.unique()
    result_dic = {}
    count = 0
    not_collected = []
    for id_house in house_unique_id:
        close_districts = df_house.loc[id_house][house_neighboor_col]
        house_long = df_house.loc[id_house][house_longitude_col]
        house_lat = df_house.loc[id_house][house_latitude_col]
        house_loc = (house_lat, house_long)
        max_distance = max_distance
        distances = []
        try:
            sub_loc_df = loc_df[loc_df['subdistrict'].isin(close_districts)]
            for i in (sub_loc_df.index):
                loc_long = sub_loc_df.loc[i][loc_longitude_col]
                loc_lat = sub_loc_df.loc[i][loc_latitude_col]
                loc_to = (loc_lat,loc_long)
                distance = distance_haver(house_loc, loc_to) 
                if distance > max_distance:
                    pass
                else:
                    distances.append(distance)    
            result_dic[id_house] = distances
        except:
            not_collected.append(id_house)
            print(house_loc)
        if count % 5000 == 0:
            print(count)
            print(np.round(count/1000))
        count += 1
            
    print(f'Number of observation with not collected data is: {len(not_collected)}')
    return result_dic, not_collected

*farmacy*

In [255]:
pharmacy_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', pharmacy_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,pharmacy_distance, 'pharmacy')

0
0.0
(41.3940535, 2.1643136)
(41.3907597, 2.1659146)
(41.4001204, 2.1613352)
(41.3938149, 2.1766059)
(41.3972152, 2.1649501)


KeyboardInterrupt: 

*hospital and cliniques*

In [256]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
hospital_and_cliniques_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', hospital_and_cliniques_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,hospital_and_cliniques_distance, 'hospital_and_cliniques')

0
0.0
(41.3853028, 2.1615237)
(41.3898395, 2.1476691)
(41.3874855, 2.1525175)
(41.3834594, 2.1610069)
5000
5.0
10000
10.0
15000
15.0
Number of observation with not collected data is: 4
Done


**IMMIGRATION RATE**

In [161]:
resource_id =  "fee0ae5c-a5d9-4e9a-8535-64bfea96d8e6"
immigration_df = from_web_api(resource_id)
immigration_df['Nombre'] = immigration_df['Nombre'].astype(float)
immigration_df['Perc'] = immigration_df['Nombre'].astype(float) / 1000
immigration_df['Codi_Barri'] = immigration_df['Codi_Barri'].astype(int)
immigration_df['subdistrict'] = immigration_df['Codi_Barri'].map(dic_codi_barri)
immigration_df = immigration_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri') 
immigration_df['Perc'] = immigration_df['Perc'] * immigration_df['WA_share']
immigration_df = immigration_df.groupby('subdistrict').sum()
immigration_df.reset_index(inplace=True)
immigration_df.head(3)
if save_sustainable_variables:
    immigration_df.to_excel(area_path+"migration"+'.xlsx')

**EMMIGRATION RATE**

In [162]:
area_path = r"C:\Users\niels\python\UPF\Thesis\Area_stats\\"
resource_id = "22d2e9f1-488b-4bfd-92b6-9b831384edd1"
emmigration_df = from_web_api(resource_id)
emmigration_df['Nombre'] = emmigration_df['Nombre'].astype(float)
emmigration_df['Perc'] = emmigration_df['Nombre'].astype(float) / 1000
emmigration_df['Codi_Barri'] = emmigration_df['Codi_Barri'].astype(int)
emmigration_df['subdistrict'] = emmigration_df['Codi_Barri'].map(dic_codi_barri)
emmigration_df = emmigration_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri') 
emmigration_df['Perc'] = emmigration_df['Perc'] * emmigration_df['WA_share']
emmigration_df = emmigration_df.groupby('subdistrict').sum()
emmigration_df.reset_index(inplace=True)
emmigration_df.head(3)
if save_sustainable_variables:
    emmigration_df.to_excel(area_path+"emigration"+'.xlsx')

**MORTALITY RATE**

In [37]:
area_path = r"C:\Users\niels\python\UPF\Thesis\Area_stats\\"
resource_id = "ac9e07cd-5328-478a-8e73-f448a17bbaee"
mortality_df = from_web_api(resource_id)
mortality_df['Nombre'] = mortality_df['Nombre'].astype(float)
mortality_df['Perc'] = mortality_df['Nombre'].astype(float) / 1000
mortality_df['Codi_Barri'] = mortality_df['Codi_Barri'].astype(int)
mortality_df['subdistrict'] = mortality_df['Codi_Barri'].map(dic_codi_barri)
mortality_df= mortality_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri') 
mortality_df['Perc'] = mortality_df['Perc'] * mortality_df['WA_share']
mortality_df= mortality_df.groupby('subdistrict').sum()
mortality_df.reset_index(inplace=True)
mortality_df.head(3)
if save_sustainable_variables:
    mortality_df.to_excel(area_path+"mortality_rate"+'.xlsx')

NameError: name 'inhabitants_subdistrict_df' is not defined

**BIRTH RATE**

In [168]:
resource_id = "cb57b490-5668-4d25-a889-a613c6c033d8"
birth_rate_df = from_web_api(resource_id)
birth_rate_df['Nombre'] = birth_rate_df['Nombre'].astype(float)
birth_rate_df['Perc'] = birth_rate_df['Nombre'].astype(float) / 1000
birth_rate_df['Codi_Barri'] = birth_rate_df['Codi_Barri'].astype(int)
birth_rate_df['subdistrict'] = birth_rate_df['Codi_Barri'].map(dic_codi_barri)
birth_rate_df = birth_rate_df .merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri') 
birth_rate_df['Perc'] = birth_rate_df['Perc'] * birth_rate_df['WA_share']
birth_rate_df  = birth_rate_df.groupby('subdistrict').sum()
birth_rate_df.reset_index(inplace=True)
birth_rate_df.head(3)
if save_sustainable_variables:
    birth_rate_df.to_excel(area_path+"birth_rate"+'.xlsx')

**POPULATION DENSITY**

In [156]:
resource_id = "540e48d8-c432-43df-b3ba-a0cf009b90ef"
density_df = from_web_api(resource_id)
print(density_df.columns)
density_df = density_df[[ 'Any','Nom_Barri','Codi_Barri','Densitat (hab/ha)','Densitat neta (hab/ha)']]
density_df['Densitat (hab/ha)'] = density_df['Densitat (hab/ha)'].astype(float)
density_df['Densitat neta (hab/ha)'] = density_df['Densitat neta (hab/ha)'].astype(float)
density_df['Codi_Barri'] = density_df['Codi_Barri'].astype(int)
density_df['subdistrict'] = density_df['Codi_Barri'].map(dic_codi_barri)
density_df = density_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri')
density_df['Densitat (hab/ha)'] = density_df['Densitat (hab/ha)'] * density_df['WA_share']
density_df['Densitat neta (hab/ha)'] = density_df['Densitat neta (hab/ha)']  * density_df['WA_share']
density_df = density_df.groupby('subdistrict').sum()
density_df.reset_index(inplace=True)
density_df.head()
if save_sustainable_variables:
    density_df.to_excel(area_path+"population_density"+'.xlsx')

Index(['Superfície Residencial (ha)', 'Codi_Districte',
       'Densitat neta (hab/ha)', 'Nom_Districte', 'Nom_Barri',
       'Densitat (hab/ha)', 'Codi_Barri', 'Població', '_id', 'Superfície (ha)',
       'Any'],
      dtype='object')


,subdistrict,Codi_Barri,Densitat (hab/ha),Densitat neta (hab/ha),WA_share
0,Subdistrict Baró de Viver,58,115.000000,673.000000,1.0
1,Subdistrict Can Baró,34,240.000000,645.000000,1.0
2,Subdistrict Can Peguera - El Turó de la Peira,93,420.057215,1102.527619,1.0
3,Subdistrict Canyelles,49,86.000000,620.000000,1.0
4,Subdistrict Ciutat Meridiana - Torre Baró - Va...,165,214.643235,565.816596,1.0


**SOCIAL COHESION**

In [175]:
url = 'https://github.com/NielsUPF/Thesis-MIIS-The-Price-Impact-of-Sustainability-on-Housing-Prices-in-Barcelona/blob/9caeaea39daeab9e8778d298075c372d6f7fa9c1/Data/Sus_Var_Creation/2018_cohesio_densitat.gpkg'

response = requests.get(url)
with open("2018_cohesio_densitat.gpkg", "wb") as file:
    file.write(response.content)

social_cohesion = gpd.read_file("2018_cohesio_densitat.gpkg")
social_cohesion = gpd.read_file(BASE_ROUT_MAP+"2018_cohesio_densitat.gpkg")
social_cohesion_df = pd.DataFrame(social_cohesion)
print(social_cohesion_df.columns)
social_cohesion_df.head(3)

Index(['id', 'co_social_', 'geometry'], dtype='object')


,id,co_social_,geometry
0,4356,10.0,"POLYGON ((423073.871 4581910.844, 423102.738 4..."
1,5785,9.0,"POLYGON ((423853.294 4586560.844, 423882.161 4..."
2,5949,9.0,"POLYGON ((423939.896 4586610.844, 423968.764 4..."


In [365]:
social_cohesion_gpkg= check_in_multipolygon(data_long_lat, 'longitude', 'latitude', social_cohesion_df, 'co_social_')
if save_sustainable_variables:
    result_to_dic(gpkp_path,social_cohesion_gpkg, 'social_cohesion')

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
Done



def check_in_multipolygon(longitude, latitude, df_gpkg):
    # Convert latitude and longitude to UTM
    easting, northing, zone_number, zone_letter = utm.from_latlon(latitude, longitude)
    point= Point(easting,northing)
    print(point)
    count  = 0
    inside = False
    for multipolygon in df_gpkg.geometry.unique():
        print(type(multipolygon))
        print(multipolygon)
        # Check if the Point is inside the Multipolygon
        #print(multipolygon)
        if multipolygon.contains(point):
            print("The point is inside this Multipolygon.")
            count += 1
            inside = True
        else:
            pass
            #print("The point is outside this Multipolygon.")
    return inside, count, latitude, longitude 

POLYGON ((423073.8706571395 4581910.84378986, 423102.738170599 4581960.84378986, 423160.473197518 4581960.84378986, 423189.34071097744 4581910.84378986, 423160.473197518 4581860.84378986, 423102.738170599 4581860.84378986, 423073.8706571395 4581910.84378986))
POLYGON ((423073.8706571395 4581910.84378986, 423102.738170599 4581960.84378986, 423160.473197518 4581960.84378986, 423189.34071097744 4581910.84378986, 423160.473197518 4581860.84378986, 423102.738170599 4581860.84378986, 423073.8706571395 4581910.84378986))

house_id = data.index[:300].values
latitude_list = data.latitude[:300].values
longitude_list = data.longitude[:300].values
latitude_social_cohesion = []
longitude_social_cohesion = []
for i in range(len(latitude_list)):
    if i%100 == True:
        print(i)
    latitude = latitude_list[i]
    longitude = longitude_list[i]
    social_cohesion_result = check_in_multipolygon(longitude,latitude, social_cohesion_df)
    if  social_cohesion_result[0] == True:
        latitude_social_cohesion.append(social_cohesion_result[2])
        longitude_social_cohesion.append(social_cohesion_result[3])

#easting, northing, zone_number, zone_letter = utm.from_latlon(latitude, longitude)
def check_in_multipolygon(longitude, latitude, df_gpkg):
    # Convert latitude and longitude to UTM
    easting, northing, zone_number, zone_letter = utm.from_latlon(latitude, longitude)
    point= Point(easting,northing)
    count  = 0
    inside = Falselatitude_list = data.latitude.values
longitude_list = data.longitude.values
latitude_vulnerable_to_heat_impact = []
longitude_vulnerable_to_heat_impact = []
latitude_density_to_heat_impact = []
longitude_density_to_heat_impact = []
latitude_people_furthest_from_green_space = []
longitude_people_furthest_from_green_space = []
for i in range(len(latitude_list)):
    if i%1000 == True:
        print(i)
    latitude = latitude_list[i]
    longitude = longitude_list[i]
    vulnerable_to_heat_impact_result = check_in_multipolygon(longitude,latitude, vulnerable_to_heatimpact_df)
    if vulnerable_to_heat_impact_result[0] == True:
        latitude_vulnerable_to_heat_impact.append( vulnerable_to_heat_impact_result[2])
        longitude_vulnerable_to_heat_impact.append( vulnerable_to_heat_impact_result[3])
    density_to_heat_impact_result = check_in_multipolygon(longitude,latitude, density_to_heatimpact_df)
    if density_to_heat_impact_result[0] == True:
        latitude_density_to_heat_impact.append( density_to_heat_impact_result[2])
        longitude_density_to_heat_impact.append( density_to_heat_impact_result[3])
    people_furthest_from_green_space_result = check_in_multipolygon(longitude,latitude, people_furthest_from_green_space_df)
    if people_furthest_from_green_space_result[0] == True:
        latitude_people_furthest_from_green_space.append( people_furthest_from_green_space_result[2])
        longitude_people_furthest_from_green_space.append( people_furthest_from_green_space_result[3])
    for multipolygon in df_gpkg.geometry.unique():
        # Check if the Point is inside the Multipolygon
        #print(multipolygon)
        if multipolygon.contains(point):
            print("The point is inside this Multipolygon.")
            count += 1
            inside = True
        else:
            pass
            #print("The point is outside this Multipolygon.")
    return inside, count, latitude, longitude 


**SHOPPING GALLERIES**

In [257]:
resource_id = "bdfb9d2f-4f72-43d4-bcb9-760523c74917"
shopping_gallery_loc_df = from_web_api(resource_id)
columns = ['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']
shopping_gallery_loc_df= shopping_gallery_loc_df[columns]
shopping_gallery_loc_df['addresses_neighborhood_id'] = shopping_gallery_loc_df['addresses_neighborhood_id'].astype(float)
shopping_gallery_loc_df['geo_epgs_4326_y'] = shopping_gallery_loc_df['geo_epgs_4326_y'].astype(float)
shopping_gallery_loc_df['geo_epgs_4326_x'] = shopping_gallery_loc_df['geo_epgs_4326_x'].astype(float)
shopping_gallery_loc_df['subdistrict'] = shopping_gallery_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
print(shopping_gallery_loc_df.secondary_filters_name.unique())
shopping_gallery_loc_df.head(3)

['Galeries comercials']


,register_id,name,institution_name,secondary_filters_name,addresses_neighborhood_id,geo_epgs_4326_x,geo_epgs_4326_y,subdistrict
0,﻿92086021583,Galeria Comercial Turó,,Galeries comercials,26.0,41.394976,2.142642,Subdistrict Sant Gervasi - Galvany
1,﻿92086021585,Galeria Comercial Sarrià,,Galeries comercials,23.0,41.399457,2.121950,Subdistrict Sarrià
2,﻿92086025558,Galeria Comercial Les Galeries Maldà,,Galeries comercials,2.0,41.383602,2.173167,Subdistrict El Gòtic


In [258]:
shopping_gallery_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', shopping_gallery_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,shopping_gallery_distance, 'shopping_gallery')

0
0.0
5000
5.0
10000
10.0
15000
15.0
Number of observation with not collected data is: 0
Done


**SHOPPING CENTRES**

In [259]:
resource_id = "d9153b84-2694-4c4f-8805-c861d4e98863"
shopping_centre_loc_df = from_web_api(resource_id)
columns = ['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']
shopping_centre_loc_df= shopping_centre_loc_df[columns]
shopping_centre_loc_df['addresses_neighborhood_id'] = shopping_centre_loc_df['addresses_neighborhood_id'].astype(float)
shopping_centre_loc_df['geo_epgs_4326_y'] = shopping_centre_loc_df['geo_epgs_4326_y'].astype(float)
shopping_centre_loc_df['geo_epgs_4326_x'] = shopping_centre_loc_df['geo_epgs_4326_x'].astype(float)
shopping_centre_loc_df['subdistrict'] = shopping_centre_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
print(shopping_centre_loc_df.secondary_filters_name.unique())
shopping_centre_loc_df.head(3)

['Grans centres comercials']


,register_id,name,institution_name,secondary_filters_name,addresses_neighborhood_id,geo_epgs_4326_x,geo_epgs_4326_y,subdistrict
0,﻿75990037167,El Corte Inglés *Plaça Catalunya,,Grans centres comercials,7.0,41.387834,2.171224,Subdistrict La Dreta de l'Eixample
1,﻿99400266477,Botiga FNAC *Arenas de Barcelona,,Grans centres comercials,9.0,41.376038,2.149975,Subdistrict La Nova Esquerra de l'Eixample
2,﻿99400230868,Botiga Wala,,Grans centres comercials,13.0,41.361781,2.137598,Subdistrict La Marina del Port


In [260]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
shopping_centre_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km',shopping_centre_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,shopping_centre_distance, 'shopping_centre')

0
0.0
5000
5.0
10000
10.0
15000
15.0
Number of observation with not collected data is: 0
Done


**LARGE ESTABALISHMENTS**

In [261]:
resource_id = "87b801a9-4747-4e20-90c9-f7152b3ed09d"
estabalishment_loc_df = from_web_api(resource_id)
columns = ['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']
estabalishment_loc_df= estabalishment_loc_df[columns]
estabalishment_loc_df['addresses_neighborhood_id'] = estabalishment_loc_df['addresses_neighborhood_id'].astype(float)
estabalishment_loc_df['geo_epgs_4326_y'] = estabalishment_loc_df['geo_epgs_4326_y'].astype(float)
estabalishment_loc_df['geo_epgs_4326_x'] = estabalishment_loc_df['geo_epgs_4326_x'].astype(float)
estabalishment_loc_df['subdistrict'] = estabalishment_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
print(estabalishment_loc_df.secondary_filters_name.unique())
estabalishment_loc_df.head(3)

['Grans establiments']


,register_id,name,institution_name,secondary_filters_name,addresses_neighborhood_id,geo_epgs_4326_x,geo_epgs_4326_y,subdistrict
0,﻿75990027374,Kenwood España,,Grans establiments,12.0,41.322205,2.134999,Subdistrict La Marina del Prat Vermell
1,﻿207150839,Dell España,,Grans establiments,0.0,-0.001094,-1.489468,NaN
2,﻿59130941,Delegació a Catalunya del Centro Español de De...,,Grans establiments,7.0,41.390355,2.170491,Subdistrict La Dreta de l'Eixample


In [262]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
estabalishment_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', estabalishment_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,estabalishment_distance, 'estabalishment')

0
0.0
5000
5.0
10000
10.0
15000
15.0
Number of observation with not collected data is: 0
Done


**STREET MARKETS AND FAIRS**

In [263]:
url =" https://www.bcn.cat/tercerlloc/files/llista-equipaments/opendatabcn_llista-equipaments_mercats-i-centres-comercials-csv.csv"
market_and_equipment_loc_df = from_web_url(url)
market_and_equipment_loc_df= market_and_equipment_loc_df[['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']]
print(market_and_equipment_loc_df.secondary_filters_name.unique())
market_and_equipment_loc_df['addresses_neighborhood_id'] = market_and_equipment_loc_df['addresses_neighborhood_id'].astype(float)
market_and_equipment_loc_df['geo_epgs_4326_y'] = market_and_equipment_loc_df['geo_epgs_4326_y'].astype(float)
market_and_equipment_loc_df['geo_epgs_4326_x'] = market_and_equipment_loc_df['geo_epgs_4326_x'].astype(float)
market_and_equipment_loc_df['subdistrict'] = market_and_equipment_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
market = ['Mercats municipals','Mercats i fires al carrer']
market_loc_df = market_and_equipment_loc_df[market_and_equipment_loc_df.secondary_filters_name.isin(market)]

['Grans establiments' 'Mercats i fires al carrer'
 'Grans centres comercials' 'WiFi BCN' 'Mercats municipals'
 'Galeries comercials']


In [264]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
market_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', market_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,market_distance, 'market')

0
0.0
5000
5.0
10000
10.0
15000
15.0
Number of observation with not collected data is: 0
Done


## CULTURAL

**CINEMAS, THEATRES AND CONCERT PLACES**

In [ ]:
url = "https://www.bcn.cat/tercerlloc/files/cultura/opendatabcn_cultura_cines-teatres-auditoris.csv"
cinema_theatre_concert_loc_df = from_web_url(url)
print(cinema_theatre_concert_loc_df.columns)
cinema_theatre_concert_loc_df = cinema_theatre_concert_loc_df[['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']]
print(cinema_theatre_concert_loc_df.secondary_filters_name.unique())
cinema_theatre_concert_loc_df['addresses_neighborhood_id'] = cinema_theatre_concert_loc_df['addresses_neighborhood_id'].astype(int)
cinema_theatre_concert_loc_df['geo_epgs_4326_x'] = cinema_theatre_concert_loc_df['geo_epgs_4326_x'].astype(float)
cinema_theatre_concert_loc_df['geo_epgs_4326_y'] = cinema_theatre_concert_loc_df['geo_epgs_4326_y'].astype(float)
cinema_theatre_concert_loc_df['subdistrict'] = cinema_theatre_concert_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
cinema = ['Cinemes']
cinema_loc_df = cinema_theatre_concert_loc_df[cinema_theatre_concert_loc_df.secondary_filters_name.isin(cinema)]
theatre = ['Teatres']
theatre_loc_df = cinema_theatre_concert_loc_df[cinema_theatre_concert_loc_df.secondary_filters_name.isin(theatre)]
concert = ['Auditoris']
concert_loc_df = cinema_theatre_concert_loc_df[cinema_theatre_concert_loc_df.secondary_filters_name.isin(concert)]

*Cinema*

In [ ]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
cinema_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km',cinema_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path, cinema_distance, 'cinema')

*Theatre*

In [ ]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
theatre_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km',theatre_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path, theatre_distance, 'theatre')

*Concert*

In [ ]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
concert_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km',concert_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path, concert_distance, 'concert')

**LIBRARIES AND MUSEUMS**

In [265]:
url = " https://www.bcn.cat/tercerlloc/files/cultura/opendatabcn_cultura_biblioteques-i-museus.csv"
library_studyroom_museum_loc_df = from_web_url(url)
library_studyroom_museum_loc_df = library_studyroom_museum_loc_df[['register_id','name', 'institution_name','addresses_neighborhood_id','secondary_filters_name', 'geo_epgs_4326_x','geo_epgs_4326_y']]
print(library_studyroom_museum_loc_df.secondary_filters_name.unique())
library_studyroom_museum_loc_df['addresses_neighborhood_id'] = library_studyroom_museum_loc_df['addresses_neighborhood_id'].astype(float)
library_studyroom_museum_loc_df['geo_epgs_4326_y'] = library_studyroom_museum_loc_df['geo_epgs_4326_y'].astype(float)
library_studyroom_museum_loc_df['geo_epgs_4326_x'] = library_studyroom_museum_loc_df['geo_epgs_4326_x'].astype(float)
library_studyroom_museum_loc_df['subdistrict'] = library_studyroom_museum_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
#df[df['B'].isin(values_to_subset_on)]
museum = ['Museus', 'Museus municipals']
museum_loc_df = library_studyroom_museum_loc_df[library_studyroom_museum_loc_df.secondary_filters_name.isin(museum)]
study_room = [ "Sales d'estudi", 'WiFi BCN','Universitats']
study_room_loc_df = library_studyroom_museum_loc_df[library_studyroom_museum_loc_df.secondary_filters_name.isin(study_room)]
library = ['Biblioteques','Biblioteques municipals']
library_loc_df = library_studyroom_museum_loc_df[library_studyroom_museum_loc_df.secondary_filters_name.isin(library)]

['Biblioteques' 'Museus' "Sales d'estudi" 'Museus municipals'
 'Universitats' 'Arxius municipals' 'Biblioteques municipals' 'WiFi BCN'
 'Parcs i jardins' 'Altres esports' 'Natació' 'Instal·lacions esportives']


*museum*

In [266]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
museum_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', museum_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,museum_distance, 'museum')

0
0.0
5000
5.0
10000
10.0
15000
15.0
Number of observation with not collected data is: 0
Done


*library*

In [267]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
library_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', library_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,library_distance, 'library')

0
0.0
5000
5.0
10000
10.0
15000
15.0
Number of observation with not collected data is: 0
Done


**RELIGIOUS INSTITUTIONS**

In [79]:
url =" https://www.bcn.cat/tercerlloc/files/llista-equipaments/opendatabcn_llista-equipaments_serveis-religiosos-csv.csv"
religious_loc_df = from_web_url(url)
columns = ['register_id','name', 'institution_name','secondary_filters_name','addresses_neighborhood_id', 'geo_epgs_4326_x','geo_epgs_4326_y']
print(religious_loc_df.secondary_filters_name.unique())
religious_loc_df = religious_loc_df[columns]
religious_loc_df['addresses_neighborhood_id'] = religious_loc_df['addresses_neighborhood_id'].astype(float)
religious_loc_df['geo_epgs_4326_y'] = religious_loc_df['geo_epgs_4326_y'].astype(float)
religious_loc_df['geo_epgs_4326_x'] = religious_loc_df['geo_epgs_4326_x'].astype(float)
religious_loc_df['subdistrict'] = religious_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
religion = ['Esglésies evangèliques', 'Esglésies catòliques',
 'Comunitats ortodoxes' ,'Sinagogues', 'Mesquites'
 "Capelles de l'Església de Jesucrist dels S.D.D." ,
           'Esglésies adventistes' ,
 'Centres taoistes', 'Centres budistes', 'Temples sikhs',
 'Centres hindús' "Centres Baha'is"]
religion_loc_df = religious_loc_df[religious_loc_df.secondary_filters_name.isin(religion)]

[nan 'Esglésies catòliques' 'Esglésies evangèliques' 'Centres budistes'
 'Mesquites' 'Temples sikhs' 'Altres' 'Biblioteques'
 "Capelles de l'Església de Jesucrist dels S.D.D." 'Centres hindús'
 'Salons del regne' 'Esglésies adventistes' "Residències d'estudiants"
 'Museus' 'Sinagogues' "Centres Baha'is" 'Centres taoistes' 'Universitats'
 'Comunitats ortodoxes']


In [89]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
religion_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', religion_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,religion_distance, 'religion')

31702050
0.0
33148486
39158016
40351755
93970881
84114683
84144318
84809865
85156127
85475409
85723149
85887505
86073242
86943143
81848703
87224917
87455193
88324275
89252787
88592049
88992972
89423193
89777295
90357468
90878934
39758183
81137028
81669178
40348404
91262603
91606238
91796091
92015334
92291997
92516308
92517374
92605182
92670506
Not collected is: (41.3912499, 2.159771)
92734746
92786494
92853686
92910516
93252066
93303890
93325259
89266795
93382762
93660221
93777323
93786023
93935303
86578563
93938123
94440733
94449239
94445515
94496557
94619773
94703993
94751957
94780085
93938118
94898225
94911838
94911981
95086942
95129290
95142185
92060679
40451747
95370105
95723191
95735860
95735948
95735966
95735969
95735981
95735992
95735993
94619767
95775989
95782552
95953958
96042262
96090541
96174751
96141584
96199564
96202066
92668801
96243712
96261595
96288705
88976071
95148649
96397108
96399598
96420780
96428372
95768690
96434153
96444314
96467730
96477418
92914729
89456641
9

100858069
100859952
100860241
100847397
100864624
100865668
100865988
100866284
100866302
100868270
100871565
100872018
100874994
100875655
100875987
99265489
100877322
100877249
100866355
100887353
100887848
100888799
100888801
100889218
100890234
100895916
100898237
100898348
100903420
89256833
100907125
100863016
100910321
100910304
100910804
100911229
95848996
100918097
100918113
100922957
100924090
100924557
100928221
100930803
100934215
100935036
100935295
100938145
100939002
100939572
100941740
100902345
100946770
100947138
100947244
100948347
100948960
100949286
100951668
100952221
100957525
100958292
100959384
100944853
100961882
100941742
100966076
100966458
100967176
100969924
100973274
100977046
100977081
100972548
100978181
100978537
100980006
100981633
99344377
100984201
100756188
100992153
100992260
100992756
97562473
100969313
101005093
99930755
101006945
101007110
101007100
101008626
101009687
101014002
101017402
100565660
100976269
101020855
101022190
101023041
101026

101097834
101097802
101015188
101102780
101102878
101103121
101105965
101106712
101107823
101107984
101109224
101109330
101108225
101110306
101111749
101112160
101114722
101117644
101118353
101124831
101125238
101125588
101125706
101125795
101125887
101127312
101126796
101127167
101127390
101127804
101126034
101128183
101128374
101129110
101129722
101130057
101132194
101132195
101134924
101135696
101135690
101135713
29944834
40010620
92362291
86438938
86555930
87303192
88318517
37501709
88546306
88563252
88590770
88939706
89062307
90591504
91261675
91927568
91927581
92281196
92278815
92553286
92670274
92949765
93242845
93510240
93587716
88822604
94471769
94633029
94704699
94731726
94763132
94934200
94991975
89315798
95344637
95356306
39500469
95738260
95738119
95738371
95738435
95773538
95798099
95921712
96041110
96151573
96197895
96447678
96456472
96603971
96633997
90242402
39807340
96661708
96745351
96745353
96782242
96871365
97086941
97151024
97170240
97200401
97350387
97479756
9750

100073143
100088000
100091713
100139115
100161923
100178334
100187879
100187896
100206248
100208061
100208062
100208023
100208025
100208076
100208068
100217680
100246608
100269806
100265867
100290426
100280866
100294140
100318837
100322697
100331851
99433242
100335047
100336539
100296249
100353156
100365461
100366792
100370830
100372695
100374263
100382570
100392587
100371299
100413770
100414360
100415044
100418445
100281979
100424150
100424276
100427849
92895950
100196123
100435613
100438968
100445122
100449701
100453724
100476848
100480569
100487683
100491495
100480327
100519186
100522361
100526547
100526861
100526008
100522064
100526202
100528843
100542140
100554301
100564634
100564673
100565099
100567424
100557061
99081450
100566423
100583273
100583369
100588142
100588190
100580843
100605004
100609142
100610139
100611113
100615744
100616940
100617893
100623017
100634064
100644168
100644318
100645420
100645773
99824860
99824864
99824899
99824900
100087222
100087225
100087269
9998234

101072028
101100743
101101353
101096093
101103408
101107564
101107744
98252480
101109824
101112718
101118754
101113907
101127304
101127959
101131047
101130985
101131598
101133021
101133227
101133867
101135683
1644595
40136609
40422364
83308716
84485794
27302657
85992378
86432286
86504432
86670068
86689523
86983056
84119197
95350553
85496956
88456465
88553412
88553389
88598794
88913565
88964433
83390846
89414221
93327174
85620292
89954346
90246954
90134796
90828546
90838456
94447552
91087310
91102556
91320458
91524589
91550994
91104732
91965341
91966170
91966700
91967187
91967487
87565876
92346120
92377661
92457904
92547357
92569569
92816320
90451019
93061260
93213136
93351919
93431281
93498894
93498837
93576945
90131944
93750053
94124156
86578745
92230138
90106516
94335946
94138285
94386740
94456713
94517730
93452963
94693672
94754127
94821518
94829757
94840137
32377804
94978062
95008964
95120624
95198792
95214361
95261992
95310124
37975296
95342894
95360663
88598724
93845999
86628783


...

100012777
100016353
100018689
100020444
100022594
100024237
100024230
98527409
100016226
100031385
100041614
99445753
100044072
97077927
100051997
100065563
96508287
100067934
100071042
100073857
97011736
100084057
100086408
100086623
100087177
100089568
100091620
100095537
100096480
100098771
100102038
100102073
100102434
100104771
100106814
100114685
100116729
99979245
100123801
100124766
96348382
100137433
100140087
100143908
100144938
94825549
100161745
100175883
100175834
100180845
98482376
99638462
100186836
100198900
100202695
100205947
100207024
100207330
100208863
100213511
100217913
100218723
100221710
100225287
100231118
100239243
100250815
100255987
87312079
99972606
100261271
100260972
99102362
100267907
96723064
100273559
100275339
100279749
100284281
100286147
100289198
100292673
100293239
100299642
100302866
100300870
97947409
100290452
100309538
100321834
100321918
100325031
100325359
100329232
100050998
100336326
100336641
100337661
100331639
100342212
100348588
10034

98256177
97809862
98268878
98268959
98268973
98269074
98271536
98271691
98272366
98273358
98292271
98293602
98321273
98321540
98324138
98340126
98351412
98368858
98393895
98388313
98399529
98416673
98422227
97923122
98121897
95598831
94583021
98032866
98196660
96831242
97024615
98444474
98474560
98516247
98530188
98561916
98570932
98571649
98578620
98186275
92569732
98583855
98677902
98686159
98708832
98716839
98308717
98746997
98748105
98140060
98803548
98674909
84186316
84186334
84186339
98831350
98835244
98846211
98847944
98853163
96612440
98916337
98336490
98929580
98954769
98274348
98999641
96899922
99002785
99003935
99004191
97902589
99022981
99023356
99023691
99027325
99028506
99035743
99036679
99044042
99049201
99062496
99062489
99071728
99077925
99088478
99094437
99099982
98926226
99104904
99105532
99109196
99122133
99124801
99133726
99151179
99155179
99169199
98729586
99202088
99251977
99263501
99268868
99317936
99330670
99358411
99360246
99389699
98427124
99393087
99402619
9

101111621
98958317
97232420
101121347
101125663
101129700
101129674
101130616
34234916
81486361
82134517
37113852
81916455
89997048
93323893
94441095
94484491
93196175
94856579
95211076
95211086
93350643
95630470
95723907
95770372
95914470
95989948
96100317
39032531
96185405
96351947
96521461
95771759
96176935
96652688
82606945
84998475
96840171
96870083
96911814
96961597
96961793
92161807
92669358
97134585
97154701
97344843
97392806
97432589
92044718
97478515
96071277
97553731
97553724
97566950
97613131
96881932
97691292
96238263
97835287
97841056
97929998
97951820
98013898
97546688
98115785
98116963
98171986
98239702
98268685
98271002
98213494
98286444
98428100
95202109
98232639
98373849
86954951
97107255
97108287
98524252
98561646
98601303
98660052
98735070
98751103
98764146
98767434
98792554
98824396
98830395
98840031
98877821
98917085
98922088
98928508
98954985
98988681
99007513
99073672
99075923
99091056
99096835
98041702
99108017
99141375
99144900
99204996
99200641
99253453
9931

97730078
97731589
97745089
97748986
97748933
97748967
97748945
97748971
97748969
97748764
97748713
97748797
97748884
97792868
97792870
97829119
97801983
97888056
97890322
97907706
97964858
97966175
97967100
97985360
97998318
98015022
98015065
98015094
98015095
98015093
98015105
98019586
98071503
98088411
98098710
98125321
96564817
98203459
98221534
98266539
98281778
98308780
98343772
98376003
98403525
98413066
98440730
98442246
97993534
97731178
97746173
98475348
98572749
98588176
98628278
88298693
98740756
98746198
98752269
98752729
98771007
98790732
98791321
98834560
98842400
95646598
98866929
98843387
98887888
98937727
98970462
92929652
98976865
98996229
99058463
99058896
99117936
99117979
99118119
99118155
99118226
99118377
99118404
99118389
99118379
99118391
99119096
99119134
99119216
98841480
99210426
99230385
99232066
99240660
99241409
99242301
99242275
99242318
99242202
99244605
99087685
99259813
99259840
99259795
99259839
99259821
99259847
99265507
99267258
99289895
99297057
9

101118750
101131181
101131541
101131529
101131658
101132533
101132885
101132939
101133189
101133425
101133624
101134276
101134305
101134433
101135260
101135697
101135655
101134507
101137418
Number of observation with not collected data is: 1
Done


**CULTURAL POINTS OF INTEREST**

In [93]:
resource_id = "31431b23-d5b9-42b8-bcd0-a84da9d8c7fa"
poi_culture_loc_df = from_web_api(resource_id)
columns = ['register_id','name', 'institution_name','addresses_neighborhood_id','geo_epgs_4326_x','geo_epgs_4326_y']
poi_culture_loc_df = poi_culture_loc_df[columns]
poi_culture_loc_df['addresses_neighborhood_id'] = poi_culture_loc_df['addresses_neighborhood_id'].astype(float)
poi_culture_loc_df['geo_epgs_4326_y'] = poi_culture_loc_df['geo_epgs_4326_y'].astype(float)
poi_culture_loc_df['geo_epgs_4326_x'] = poi_culture_loc_df['geo_epgs_4326_x'].astype(float)
poi_culture_loc_df['subdistrict'] = poi_culture_loc_df['addresses_neighborhood_id'].map(dic_codi_barri)
poi_culture_loc_df.head(3)

,register_id,name,institution_name,addresses_neighborhood_id,geo_epgs_4326_x,geo_epgs_4326_y,subdistrict
0,﻿92086015836,Parc de l'Espanya Industrial,,18.0,41.376536,2.139874,Subdistrict Sants
1,﻿92259113820,Escultura Mistos,,41.0,41.430845,2.151482,Subdistrict La Vall d'Hebron - La Clota
2,﻿93308093528,Centre Comercial L'Illa Diagonal,,19.0,41.389746,2.135396,Subdistrict Les Corts


In [94]:
distance_path = r"C:\Users\niels\python\UPF\Thesis\Distance\\"
poi_culture_distance = distance_house(10, data_long_lat,'longitude','latitude','neighboors_10km', poi_culture_loc_df,"geo_epgs_4326_y","geo_epgs_4326_x")
if save_sustainable_variables:
    result_to_dic(distance_path,poi_culture_distance, 'poi_culture')

1.0
1.0
1.0
1.0
Number of observation with not collected data is: 0
Done


## ECONOMIC-FINANCIAL

In [198]:
#income distribution area
gini_index_df = from_web_api('44dd0cb5-41ef-4359-a1c5-9b3359347ebb')
p80_p20_income_distribution_df = from_web_api('5be3a337-647c-407f-ad73-4b6859540f90')
bruto_household_income_df = from_web_api('41b26bd2-a570-4275-acaf-510d04c36843')
bruto_person_income_df = from_web_api('e6b601e9-b0d3-422d-91db-2cdb442ec775')
disposable_income_df = from_web_api("b07003f3-d01d-4815-b399-36eb1571de65")
unemployment_weight_df = from_web_api('af79b704-ff47-48f7-a66b-8b9fd738a5f1')

#ints
gini_index_df['Codi_Barri'] = gini_index_df['Codi_Barri'].astype(int)
gini_index_df['subdistrict'] = gini_index_df['Codi_Barri'].map(dic_codi_barri)
p80_p20_income_distribution_df['Codi_Barri'] = p80_p20_income_distribution_df['Codi_Barri'].astype(int)
bruto_household_income_df['Codi_Barri'] = bruto_household_income_df['Codi_Barri'].astype(int)
bruto_person_income_df['Codi_Barri'] = bruto_person_income_df['Codi_Barri'].astype(int)
disposable_income_df['Codi_Barri'] =disposable_income_df['Codi_Barri'].astype(int)
unemployment_weight_df['Codi_Barri'] = unemployment_weight_df['Codi_Barri'].astype(int)

#floats
gini_index_df['Index_Gini'] = gini_index_df['Index_Gini'].astype(float)
p80_p20_income_distribution_df['Distribucio_P80_20'] = p80_p20_income_distribution_df['Distribucio_P80_20'].astype(float)
bruto_household_income_df['Import_Renda_Bruta_€'] = bruto_household_income_df['Import_Renda_Bruta_€'].astype(float)
bruto_person_income_df['Import_Renda_Bruta_€'] = bruto_person_income_df['Import_Renda_Bruta_€'].astype(float)
disposable_income_df['Euros_Any'] =disposable_income_df['Euros_Any'].astype(float)
unemployment_weight_df['Pes_atur'] = unemployment_weight_df['Pes_atur'].astype(float)

gini_index_df = gini_index_df.groupby('Codi_Barri').mean()
p80_p20_income_distribution_df = p80_p20_income_distribution_df.groupby('Codi_Barri').mean()
bruto_household_income_df = bruto_household_income_df.groupby('Codi_Barri').mean()
bruto_person_income_df =  bruto_person_income_df.groupby('Codi_Barri').mean()
disposable_income_df =  disposable_income_df.groupby('Codi_Barri').mean()
unemployment_weight_df = unemployment_weight_df.groupby('Codi_Barri').mean()

gini_index_df.reset_index(inplace=True)
p80_p20_income_distribution_df.reset_index(inplace=True)
bruto_household_income_df.reset_index(inplace=True)
bruto_person_income_df.reset_index(inplace=True)
disposable_income_df.reset_index(inplace=True)
unemployment_weight_df.reset_index(inplace=True)


gini_index_df = gini_index_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri')
gini_index_df['Index_Gini'] = gini_index_df['Index_Gini'] * gini_index_df['WA_share']
gini_index_df['subdistrict'] = gini_index_df['Codi_Barri'].map(dic_codi_barri)
gini_index_df = gini_index_df.groupby('subdistrict').sum()

p80_p20_income_distribution_df = p80_p20_income_distribution_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri')
p80_p20_income_distribution_df['Distribucio_P80_20'] = p80_p20_income_distribution_df['Distribucio_P80_20'] * p80_p20_income_distribution_df['WA_share']
p80_p20_income_distribution_df['subdistrict'] = p80_p20_income_distribution_df['Codi_Barri'].map(dic_codi_barri)
p80_p20_income_distribution_df= p80_p20_income_distribution_df.groupby('subdistrict').sum()

bruto_household_income_df = bruto_household_income_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri')
bruto_household_income_df['Import_Renda_Bruta_€'] = bruto_household_income_df['Import_Renda_Bruta_€'] * bruto_household_income_df['WA_share']
bruto_household_income_df['subdistrict'] = bruto_household_income_df['Codi_Barri'].map(dic_codi_barri)
bruto_household_income_df= bruto_household_income_df.groupby('subdistrict').sum()


bruto_person_income_df= bruto_person_income_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri')
bruto_person_income_df['Import_Renda_Bruta_€'] = bruto_person_income_df['Import_Renda_Bruta_€'] * bruto_person_income_df['WA_share']
bruto_person_income_df['subdistrict'] = bruto_person_income_df['Codi_Barri'].map(dic_codi_barri)
bruto_person_income_df= bruto_person_income_df.groupby('subdistrict').sum()

disposable_income_df = disposable_income_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri')
disposable_income_df ['Euros_Any'] = disposable_income_df ['Euros_Any'] * disposable_income_df['WA_share']
disposable_income_df['subdistrict'] = disposable_income_df['Codi_Barri'].map(dic_codi_barri)
disposable_income_df = disposable_income_df.groupby('subdistrict').sum()

unemployment_weight_df= unemployment_weight_df.merge(inhabitants_subdistrict_df[['Codi_Barri','WA_share']], on='Codi_Barri')
unemployment_weight_df ['Pes_atur'] = unemployment_weight_df['Pes_atur'] *unemployment_weight_df['WA_share']
unemployment_weight_df ['subdistrict'] =unemployment_weight_df['Codi_Barri'].map(dic_codi_barri)
unemployment_weight_df= unemployment_weight_df.groupby('subdistrict').sum()

gini_index_df.reset_index(inplace=True)
p80_p20_income_distribution_df.reset_index(inplace=True)
bruto_household_income_df.reset_index(inplace=True)
bruto_person_income_df.reset_index(inplace=True)
disposable_income_df.reset_index(inplace=True)
unemployment_weight_df.reset_index(inplace=True)

#gini_index_df['subdistrict'] = gini_index_df['Codi_Barri'].map(dic_codi_barri)
#p80_p20_income_distribution_df['subdistrict'] = p80_p20_income_distribution_df['Codi_Barri'].map(dic_codi_barri)
#bruto_household_income_df['subdistrict'] = bruto_household_income_df['Codi_Barri'].map(dic_codi_barri)
#bruto_person_income_df['subdistrict'] = bruto_person_income_df['Codi_Barri'].map(dic_codi_barri)
#disposable_income_df['subdistrict'] = disposable_income_df['Codi_Barri'].map(dic_codi_barri)
#unemployment_weight_df['subdistrict'] = unemployment_weight_df['Codi_Barri'].map(dic_codi_barri)
#p80_p20_income_distribution_df = p80_p20_income_distribution_df.groupby('Codi_Barri').mean()
#bruto_household_income_df = bruto_household_income_df.groupby('Codi_Barri').mean()
#bruto_person_income_df =  bruto_person_income_df.groupby('Codi_Barri').mean()
#disposable_income_df =  disposable_income_df.groupby('Codi_Barri').mean()
#unemployment_weight_df = unemployment_weight_df.groupby('Codi_Barri').mean()

#to excel
if save_sustainable_variables:
    gini_index_df.to_excel(area_path+"gini_index"+'.xlsx')
    p80_p20_income_distribution_df.to_excel(area_path+"p80_p20_income_distribution"+'.xlsx')
    bruto_household_income_df.to_excel(area_path+"bruto_household_income"+'.xlsx')
    bruto_person_income_df.to_excel(area_path+"bruto_person_income"+'.xlsx')
    disposable_income_df.to_excel(area_path+"disposable_income"+'.xlsx')
    unemployment_weight_df.to_excel(area_path+"unemployment_weight"+'.xlsx')

#income_distribution_df.head(3)



## END